# Ordering by metrics and retraining phase

## Dataset: GTSRB

## Experiment configuration 1


## Configuration 1
	1. Incremental guided retraining starting from scratch using the new adversarial inputs and original training set

In [ ]:
pip install --user tensorflow==2.5

In [1]:
import argparse

import numpy as np
import tensorflow as tf
import keras.backend as K

import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.regularizers import l2

import glob
import os
import cv2
import keras


import time
import argparse

from tqdm import tqdm

from keras.models import load_model, Model


In [2]:
tf.__version__ == '2.5.0' # Version of tensorflow

True

In [3]:
cd C:/Users/fjdur/Documents/utils/

C:\Users\fjdur\Documents\utils


In [4]:
# utils for project
import utils_guided_retraining as utils

2.5.0
keras
2.5.0


In [5]:
#cd '../notebooks/'

In [6]:
dataset = "gtsrb"

In [7]:
x_train,y_train = utils.get_data(dataset,"Train")
x_val,y_val = utils.get_data(dataset,"Val")
x_test,y_test = utils.get_data(dataset,"Test")

In [8]:
x_train_and_adversary,y_train_and_adversary = utils.get_data(dataset,"Train_and_adversary")

In [9]:
x_adversary_training = x_train_and_adversary[len(x_train):]
print(len(x_adversary_training))
y_adversary_training = y_train_and_adversary[len(y_train):]

print(len(y_adversary_training))

3921
3921


In [10]:
# Obtaining adversarial examples for testing 
x_test_and_adversary,y_test_and_adversary = utils.get_adversarial_data(dataset,'Test_adversarial') 

In [11]:
x_adversary_test_fgsm = x_test_and_adversary[len(x_test):]
print(len(x_adversary_test_fgsm))
y_adversary_test_fgsm = y_test_and_adversary[len(y_test):]

print(len(y_adversary_test_fgsm))

3921
3921


## ----

In [12]:
# Original model 
model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/"


In [13]:
model_original = utils.My_model(dataset,True, model_dir)

Model loaded correctly


In [14]:
lsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/lsa_values.npy"
dsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/dsa_values.npy"

In [14]:
n_data_points = 1750

## Loading LSA and DSA values 

In [15]:
lsa_values = np.load(lsa_direction)


In [52]:
dsa_values = np.load(dsa_direction)

In [17]:

# Obtaining top n images by LSA values
top_images_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),x_train_and_adversary)
top_labels_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),y_train_and_adversary)


In [53]:

top_images_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),x_train_and_adversary)
top_labels_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),y_train_and_adversary)


In [18]:
print(x_train_and_adversary.shape)
len(top_images_by_lsa)//20

(35287, 48, 48, 1)


1764

In [19]:
m = 1750 
n = 0
image_sets_lsa = []
label_sets_lsa = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_lsa)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_lsa)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_lsa)%m))
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m])
    image_sets_lsa.append(top_images_by_lsa_n)
    label_sets_lsa.append(top_labels_by_lsa_n)
    print(len(top_images_by_lsa_n))
    n += m

0 :
0  ->  1750
1750
1 :
0  ->  3500
3500
2 :
0  ->  5250
5250
3 :
0  ->  7000
7000
4 :
0  ->  8750
8750
5 :
0  ->  10500
10500
6 :
0  ->  12250
12250
7 :
0  ->  14000
14000
8 :
0  ->  15750
15750
9 :
0  ->  17500
17500
10 :
0  ->  19250
19250
11 :
0  ->  21000
21000
12 :
0  ->  22750
22750
13 :
0  ->  24500
24500
14 :
0  ->  26250
26250
15 :
0  ->  28000
28000
16 :
0  ->  29750
29750
17 :
0  ->  31500
31500
18 :
0  ->  33250
33250
19 :
0  ->  35000
35000
20 :
Last
0  ->  37037
35287


In [ ]:
m = 1750
n = 0
image_sets_lsa = []
label_sets_lsa = []


for i in range(len(top_images_by_lsa)//m):
    print(i,":")
    if (i+1 >= len(top_images_by_lsa)//m):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_lsa)%m))
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m])
    image_sets_lsa.append(top_images_by_lsa_n)
    label_sets_lsa.append(top_labels_by_lsa_n)
    print(len(top_images_by_lsa_n))
    n += m

print(len(image_sets_lsa)," total sets")

## Training guided by LSA

In [44]:
len(label_sets_lsa)

21

In [20]:
print(model_dir)

models_lsa = []
for i in range(len(label_sets_lsa)):
    print(i,":")
    model = utils.My_model(dataset,False,None)
    model.compile_model()
    models_lsa.append(model)


C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/
0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled
20 :
Model compiled


In [21]:
n = 0
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n=n+1

Epoch 1/10
28/28 [==============================] - 86s 631ms/step - loss: 4.8942 - accuracy: 0.2407 - val_loss: 3.6864 - val_accuracy: 0.0283
Epoch 2/10
28/28 [==============================] - 16s 592ms/step - loss: 2.9764 - accuracy: 0.3766 - val_loss: 9.7876 - val_accuracy: 0.0283
Epoch 3/10
28/28 [==============================] - 17s 612ms/step - loss: 3.5282 - accuracy: 0.3496 - val_loss: 3.6120 - val_accuracy: 0.0283
Epoch 4/10
28/28 [==============================] - 17s 611ms/step - loss: 2.9806 - accuracy: 0.3644 - val_loss: 3.6103 - val_accuracy: 0.0283
Epoch 5/10
28/28 [==============================] - 17s 608ms/step - loss: 2.6414 - accuracy: 0.3548 - val_loss: 3.4948 - val_accuracy: 0.0832
Epoch 6/10
28/28 [==============================] - 17s 614ms/step - loss: 2.3707 - accuracy: 0.3747 - val_loss: 3.3254 - val_accuracy: 0.0952
Epoch 7/10
28/28 [==============================] - 17s 611ms/step - loss: 2.2980 - accuracy: 0.3880 - val_loss: 3.3016 - val_accuracy: 0.1386

In [22]:
n=1
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

Epoch 1/10
55/55 [==============================] - 33s 528ms/step - loss: 3.2903 - accuracy: 0.2114 - val_loss: 4.0217 - val_accuracy: 0.0283
Epoch 2/10
55/55 [==============================] - 29s 524ms/step - loss: 2.6193 - accuracy: 0.2629 - val_loss: 6.6657 - val_accuracy: 0.0311
Epoch 3/10
55/55 [==============================] - 29s 529ms/step - loss: 2.4747 - accuracy: 0.3226 - val_loss: 4.8335 - val_accuracy: 0.0595
Epoch 4/10
55/55 [==============================] - 29s 529ms/step - loss: 1.7177 - accuracy: 0.5600 - val_loss: 3.4569 - val_accuracy: 0.1253
Epoch 5/10
55/55 [==============================] - 29s 537ms/step - loss: 1.3873 - accuracy: 0.6579 - val_loss: 3.0631 - val_accuracy: 0.1699
Epoch 6/10
55/55 [==============================] - 30s 550ms/step - loss: 1.1985 - accuracy: 0.6894 - val_loss: 2.7094 - val_accuracy: 0.3154
Epoch 7/10
55/55 [==============================] - 31s 564ms/step - loss: 1.0788 - accuracy: 0.7160 - val_loss: 2.4129 - val_accuracy: 0.4172

In [23]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

2
Epoch 1/10
83/83 [==============================] - 46s 501ms/step - loss: 3.0930 - accuracy: 0.1988 - val_loss: 4.5288 - val_accuracy: 0.0383
Epoch 2/10
83/83 [==============================] - 42s 512ms/step - loss: 2.5791 - accuracy: 0.2373 - val_loss: 3.6951 - val_accuracy: 0.0676
Epoch 3/10
83/83 [==============================] - 43s 513ms/step - loss: 1.9391 - accuracy: 0.4756 - val_loss: 3.4882 - val_accuracy: 0.1653
Epoch 4/10
83/83 [==============================] - 43s 520ms/step - loss: 1.2346 - accuracy: 0.6786 - val_loss: 3.3169 - val_accuracy: 0.1756
Epoch 5/10
83/83 [==============================] - 44s 536ms/step - loss: 1.0522 - accuracy: 0.7252 - val_loss: 2.6947 - val_accuracy: 0.2626
Epoch 6/10
83/83 [==============================] - 45s 541ms/step - loss: 0.8748 - accuracy: 0.7719 - val_loss: 2.1842 - val_accuracy: 0.4111
Epoch 7/10
83/83 [==============================] - 44s 535ms/step - loss: 0.7217 - accuracy: 0.8037 - val_loss: 1.8941 - val_accuracy: 0.46

In [24]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

3
Epoch 1/10
110/110 [==============================] - 64s 535ms/step - loss: 3.8196 - accuracy: 0.1528 - val_loss: 4.9618 - val_accuracy: 0.0383
Epoch 2/10
110/110 [==============================] - 58s 527ms/step - loss: 2.6281 - accuracy: 0.2252 - val_loss: 3.7556 - val_accuracy: 0.1434
Epoch 3/10
110/110 [==============================] - 58s 527ms/step - loss: 1.3634 - accuracy: 0.6534 - val_loss: 3.2086 - val_accuracy: 0.2866
Epoch 4/10
110/110 [==============================] - 58s 526ms/step - loss: 0.8408 - accuracy: 0.7970 - val_loss: 2.5428 - val_accuracy: 0.3065
Epoch 5/10
110/110 [==============================] - 58s 527ms/step - loss: 0.7295 - accuracy: 0.8227 - val_loss: 2.7116 - val_accuracy: 0.3927
Epoch 6/10
110/110 [==============================] - 58s 525ms/step - loss: 0.6473 - accuracy: 0.8327 - val_loss: 2.1001 - val_accuracy: 0.4478
Epoch 7/10
110/110 [==============================] - 58s 530ms/step - loss: 0.5219 - accuracy: 0.8584 - val_loss: 2.0878 - val_

In [25]:
print(n) #4
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

4
Epoch 1/10
137/137 [==============================] - 75s 520ms/step - loss: 3.3948 - accuracy: 0.1220 - val_loss: 4.5539 - val_accuracy: 0.0306
Epoch 2/10
137/137 [==============================] - 70s 514ms/step - loss: 2.8313 - accuracy: 0.1594 - val_loss: 3.6048 - val_accuracy: 0.1651
Epoch 3/10
137/137 [==============================] - 71s 517ms/step - loss: 1.5666 - accuracy: 0.5983 - val_loss: 3.8482 - val_accuracy: 0.2235
Epoch 4/10
137/137 [==============================] - 71s 517ms/step - loss: 0.8933 - accuracy: 0.7749 - val_loss: 3.0433 - val_accuracy: 0.3361
Epoch 5/10
137/137 [==============================] - 71s 520ms/step - loss: 0.6461 - accuracy: 0.8356 - val_loss: 3.1534 - val_accuracy: 0.3373
Epoch 6/10
137/137 [==============================] - 72s 527ms/step - loss: 0.4980 - accuracy: 0.8668 - val_loss: 2.8683 - val_accuracy: 0.3858
Epoch 7/10
137/137 [==============================] - 72s 524ms/step - loss: 0.4277 - accuracy: 0.8856 - val_loss: 1.8118 - val_

In [26]:
n=5
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

5
Epoch 1/10
165/165 [==============================] - 89s 515ms/step - loss: 3.2702 - accuracy: 0.1030 - val_loss: 5.4055 - val_accuracy: 0.0365
Epoch 2/10
165/165 [==============================] - 88s 531ms/step - loss: 2.7814 - accuracy: 0.2058 - val_loss: 3.3709 - val_accuracy: 0.2087
Epoch 3/10
165/165 [==============================] - 85s 516ms/step - loss: 1.2557 - accuracy: 0.6691 - val_loss: 2.7994 - val_accuracy: 0.2881
Epoch 4/10
165/165 [==============================] - 86s 518ms/step - loss: 0.6436 - accuracy: 0.8399 - val_loss: 2.6982 - val_accuracy: 0.3873
Epoch 5/10
165/165 [==============================] - 86s 519ms/step - loss: 0.4664 - accuracy: 0.8788 - val_loss: 2.1067 - val_accuracy: 0.4825
Epoch 6/10
165/165 [==============================] - 86s 519ms/step - loss: 0.3986 - accuracy: 0.8957 - val_loss: 1.6208 - val_accuracy: 0.5843
Epoch 7/10
165/165 [==============================] - 85s 517ms/step - loss: 0.2914 - accuracy: 0.9210 - val_loss: 1.3937 - val_

In [27]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

6
Epoch 1/10
192/192 [==============================] - 103s 515ms/step - loss: 3.3278 - accuracy: 0.0994 - val_loss: 5.7535 - val_accuracy: 0.0799
Epoch 2/10
192/192 [==============================] - 100s 520ms/step - loss: 2.4454 - accuracy: 0.3275 - val_loss: 2.8760 - val_accuracy: 0.2896
Epoch 3/10
192/192 [==============================] - 100s 519ms/step - loss: 0.8770 - accuracy: 0.7762 - val_loss: 2.3132 - val_accuracy: 0.4376
Epoch 4/10
192/192 [==============================] - 98s 513ms/step - loss: 0.5085 - accuracy: 0.8727 - val_loss: 1.8283 - val_accuracy: 0.5507
Epoch 5/10
192/192 [==============================] - 99s 513ms/step - loss: 0.3383 - accuracy: 0.9114 - val_loss: 1.2580 - val_accuracy: 0.6474
Epoch 6/10
192/192 [==============================] - 99s 514ms/step - loss: 0.2844 - accuracy: 0.9264 - val_loss: 1.0090 - val_accuracy: 0.7402
Epoch 7/10
192/192 [==============================] - 98s 513ms/step - loss: 0.2229 - accuracy: 0.9353 - val_loss: 1.0459 - v

In [28]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

7
Epoch 1/10
219/219 [==============================] - 117s 513ms/step - loss: 3.4395 - accuracy: 0.0976 - val_loss: 4.8942 - val_accuracy: 0.0699
Epoch 2/10
219/219 [==============================] - 112s 512ms/step - loss: 2.4346 - accuracy: 0.3598 - val_loss: 3.1111 - val_accuracy: 0.3248
Epoch 3/10
219/219 [==============================] - 112s 509ms/step - loss: 0.8701 - accuracy: 0.7683 - val_loss: 2.3570 - val_accuracy: 0.4011
Epoch 4/10
219/219 [==============================] - 114s 518ms/step - loss: 0.4767 - accuracy: 0.8681 - val_loss: 1.7511 - val_accuracy: 0.5154
Epoch 5/10
219/219 [==============================] - 112s 512ms/step - loss: 0.3502 - accuracy: 0.9039 - val_loss: 1.3817 - val_accuracy: 0.6017
Epoch 6/10
219/219 [==============================] - 111s 509ms/step - loss: 0.2708 - accuracy: 0.9228 - val_loss: 1.1786 - val_accuracy: 0.6711
Epoch 7/10
219/219 [==============================] - 111s 508ms/step - loss: 0.2364 - accuracy: 0.9313 - val_loss: 1.0197

In [29]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

8
Epoch 1/10
247/247 [==============================] - 134s 525ms/step - loss: 3.4650 - accuracy: 0.0993 - val_loss: 3.8192 - val_accuracy: 0.1319
Epoch 2/10
247/247 [==============================] - 125s 505ms/step - loss: 1.8026 - accuracy: 0.5293 - val_loss: 2.2716 - val_accuracy: 0.4108
Epoch 3/10
247/247 [==============================] - 125s 506ms/step - loss: 0.5870 - accuracy: 0.8426 - val_loss: 1.6093 - val_accuracy: 0.5586
Epoch 4/10
247/247 [==============================] - 125s 506ms/step - loss: 0.3588 - accuracy: 0.9043 - val_loss: 1.3514 - val_accuracy: 0.6132
Epoch 5/10
247/247 [==============================] - 125s 506ms/step - loss: 0.2675 - accuracy: 0.9232 - val_loss: 1.1152 - val_accuracy: 0.7025
Epoch 6/10
247/247 [==============================] - 117s 475ms/step - loss: 0.2154 - accuracy: 0.9379 - val_loss: 0.9043 - val_accuracy: 0.7522
Epoch 7/10
247/247 [==============================] - 126s 508ms/step - loss: 0.1781 - accuracy: 0.9486 - val_loss: 0.9078

In [30]:
print(n)#9
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

9
Epoch 1/10
274/274 [==============================] - 144s 510ms/step - loss: 3.5127 - accuracy: 0.0856 - val_loss: 4.4511 - val_accuracy: 0.0646
Epoch 2/10
274/274 [==============================] - 137s 501ms/step - loss: 2.9176 - accuracy: 0.2180 - val_loss: 2.6420 - val_accuracy: 0.5019
Epoch 3/10
274/274 [==============================] - 139s 506ms/step - loss: 0.8819 - accuracy: 0.7561 - val_loss: 1.3994 - val_accuracy: 0.6318
Epoch 4/10
274/274 [==============================] - 138s 504ms/step - loss: 0.3823 - accuracy: 0.8957 - val_loss: 1.1394 - val_accuracy: 0.7106
Epoch 5/10
274/274 [==============================] - 138s 504ms/step - loss: 0.2385 - accuracy: 0.9330 - val_loss: 1.1456 - val_accuracy: 0.7122
Epoch 6/10
274/274 [==============================] - 138s 505ms/step - loss: 0.1898 - accuracy: 0.9458 - val_loss: 1.0317 - val_accuracy: 0.7834
Epoch 7/10
274/274 [==============================] - 138s 504ms/step - loss: 0.1632 - accuracy: 0.9519 - val_loss: 1.0692

In [32]:
## print(n)
n=10
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

Epoch 1/10
301/301 [==============================] - 139s 448ms/step - loss: 3.4556 - accuracy: 0.0968 - val_loss: 3.3064 - val_accuracy: 0.4080
Epoch 2/10
301/301 [==============================] - 137s 456ms/step - loss: 1.3402 - accuracy: 0.6322 - val_loss: 2.9244 - val_accuracy: 0.6076
Epoch 3/10
301/301 [==============================] - 144s 477ms/step - loss: 0.4939 - accuracy: 0.8632 - val_loss: 1.9233 - val_accuracy: 0.6430
Epoch 4/10
301/301 [==============================] - 151s 500ms/step - loss: 0.2888 - accuracy: 0.9201 - val_loss: 1.1837 - val_accuracy: 0.7201
Epoch 5/10
301/301 [==============================] - 153s 509ms/step - loss: 0.2208 - accuracy: 0.9382 - val_loss: 1.0221 - val_accuracy: 0.7413
Epoch 6/10
301/301 [==============================] - 154s 511ms/step - loss: 0.1796 - accuracy: 0.9505 - val_loss: 0.9081 - val_accuracy: 0.7936
Epoch 7/10
301/301 [==============================] - 152s 505ms/step - loss: 0.1501 - accuracy: 0.9562 - val_loss: 1.0083 -

In [33]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

11
Epoch 1/10
329/329 [==============================] - 171s 506ms/step - loss: 3.4186 - accuracy: 0.0854 - val_loss: 4.2187 - val_accuracy: 0.4289
Epoch 2/10
329/329 [==============================] - 166s 505ms/step - loss: 1.3492 - accuracy: 0.6424 - val_loss: 2.6102 - val_accuracy: 0.6127
Epoch 3/10
329/329 [==============================] - 165s 503ms/step - loss: 0.4619 - accuracy: 0.8796 - val_loss: 1.8617 - val_accuracy: 0.6407
Epoch 4/10
329/329 [==============================] - 165s 502ms/step - loss: 0.2683 - accuracy: 0.9279 - val_loss: 1.2021 - val_accuracy: 0.7214
Epoch 5/10
329/329 [==============================] - 166s 505ms/step - loss: 0.2077 - accuracy: 0.9433 - val_loss: 0.9294 - val_accuracy: 0.7474
Epoch 6/10
329/329 [==============================] - 165s 503ms/step - loss: 0.1871 - accuracy: 0.9485 - val_loss: 1.1209 - val_accuracy: 0.7471
Epoch 7/10
329/329 [==============================] - 165s 503ms/step - loss: 0.1484 - accuracy: 0.9599 - val_loss: 0.834

In [34]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

12
Epoch 1/10
356/356 [==============================] - 185s 509ms/step - loss: 3.4829 - accuracy: 0.0783 - val_loss: 3.9674 - val_accuracy: 0.0732
Epoch 2/10
356/356 [==============================] - 179s 502ms/step - loss: 2.3236 - accuracy: 0.4000 - val_loss: 1.9234 - val_accuracy: 0.6423
Epoch 3/10
356/356 [==============================] - 180s 505ms/step - loss: 0.5004 - accuracy: 0.8629 - val_loss: 1.2374 - val_accuracy: 0.7408
Epoch 4/10
356/356 [==============================] - 179s 502ms/step - loss: 0.2475 - accuracy: 0.9321 - val_loss: 0.6373 - val_accuracy: 0.8443
Epoch 5/10
356/356 [==============================] - 186s 522ms/step - loss: 0.1857 - accuracy: 0.9495 - val_loss: 0.5171 - val_accuracy: 0.8520
Epoch 6/10
356/356 [==============================] - 181s 508ms/step - loss: 0.1376 - accuracy: 0.9615 - val_loss: 0.6325 - val_accuracy: 0.8505
Epoch 7/10
356/356 [==============================] - 178s 501ms/step - loss: 0.1234 - accuracy: 0.9649 - val_loss: 0.481

In [35]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

13
Epoch 1/10
383/383 [==============================] - 198s 504ms/step - loss: 3.3532 - accuracy: 0.1082 - val_loss: 2.5006 - val_accuracy: 0.5292
Epoch 2/10
383/383 [==============================] - 192s 501ms/step - loss: 1.0195 - accuracy: 0.7213 - val_loss: 1.5018 - val_accuracy: 0.7038
Epoch 3/10
383/383 [==============================] - 193s 503ms/step - loss: 0.3303 - accuracy: 0.9126 - val_loss: 1.3701 - val_accuracy: 0.7459
Epoch 4/10
383/383 [==============================] - 191s 498ms/step - loss: 0.2068 - accuracy: 0.9452 - val_loss: 0.9993 - val_accuracy: 0.7831
Epoch 5/10
383/383 [==============================] - 192s 502ms/step - loss: 0.1529 - accuracy: 0.9584 - val_loss: 0.7121 - val_accuracy: 0.8227
Epoch 6/10
383/383 [==============================] - 193s 503ms/step - loss: 0.1381 - accuracy: 0.9613 - val_loss: 0.7164 - val_accuracy: 0.8443
Epoch 7/10
383/383 [==============================] - 192s 500ms/step - loss: 0.1197 - accuracy: 0.9683 - val_loss: 0.581

In [36]:
print(n)#14
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

14
Epoch 1/10
411/411 [==============================] - 209s 498ms/step - loss: 3.3909 - accuracy: 0.1109 - val_loss: 2.5643 - val_accuracy: 0.5394
Epoch 2/10
411/411 [==============================] - 207s 504ms/step - loss: 0.9343 - accuracy: 0.7444 - val_loss: 1.3228 - val_accuracy: 0.7290
Epoch 3/10
411/411 [==============================] - 213s 519ms/step - loss: 0.3169 - accuracy: 0.9153 - val_loss: 0.8572 - val_accuracy: 0.8301
Epoch 4/10
411/411 [==============================] - 217s 529ms/step - loss: 0.1838 - accuracy: 0.9505 - val_loss: 0.4850 - val_accuracy: 0.8671
Epoch 5/10
411/411 [==============================] - 196s 478ms/step - loss: 0.1428 - accuracy: 0.9594 - val_loss: 0.6948 - val_accuracy: 0.8579
Epoch 6/10
411/411 [==============================] - 204s 497ms/step - loss: 0.1184 - accuracy: 0.9659 - val_loss: 0.2915 - val_accuracy: 0.9104
Epoch 7/10
411/411 [==============================] - 205s 498ms/step - loss: 0.1115 - accuracy: 0.9701 - val_loss: 0.625

In [37]:
n=15
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

15
Epoch 1/10
438/438 [==============================] - 221s 495ms/step - loss: 3.3320 - accuracy: 0.1298 - val_loss: 1.7462 - val_accuracy: 0.6652
Epoch 2/10
438/438 [==============================] - 217s 496ms/step - loss: 0.7742 - accuracy: 0.7753 - val_loss: 0.9061 - val_accuracy: 0.7938
Epoch 3/10
438/438 [==============================] - 218s 497ms/step - loss: 0.2926 - accuracy: 0.9180 - val_loss: 0.5027 - val_accuracy: 0.9109
Epoch 4/10
438/438 [==============================] - 217s 495ms/step - loss: 0.1766 - accuracy: 0.9514 - val_loss: 0.3842 - val_accuracy: 0.9219
Epoch 5/10
438/438 [==============================] - 217s 494ms/step - loss: 0.1340 - accuracy: 0.9616 - val_loss: 0.2531 - val_accuracy: 0.9416
Epoch 6/10
438/438 [==============================] - 216s 494ms/step - loss: 0.1107 - accuracy: 0.9686 - val_loss: 0.3052 - val_accuracy: 0.9403
Epoch 7/10
438/438 [==============================] - 216s 494ms/step - loss: 0.0978 - accuracy: 0.9724 - val_loss: 0.186

In [38]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

16
Epoch 1/10
465/465 [==============================] - 236s 499ms/step - loss: 3.4119 - accuracy: 0.0977 - val_loss: 1.7727 - val_accuracy: 0.6591
Epoch 2/10
465/465 [==============================] - 233s 500ms/step - loss: 0.8846 - accuracy: 0.7451 - val_loss: 0.8356 - val_accuracy: 0.8658
Epoch 3/10
465/465 [==============================] - 231s 496ms/step - loss: 0.2877 - accuracy: 0.9173 - val_loss: 0.4265 - val_accuracy: 0.9219
Epoch 4/10
465/465 [==============================] - 230s 495ms/step - loss: 0.1667 - accuracy: 0.9533 - val_loss: 0.2314 - val_accuracy: 0.9283
Epoch 5/10
465/465 [==============================] - 230s 495ms/step - loss: 0.1188 - accuracy: 0.9668 - val_loss: 0.3376 - val_accuracy: 0.9393
Epoch 6/10
465/465 [==============================] - 231s 496ms/step - loss: 0.1058 - accuracy: 0.9705 - val_loss: 0.2148 - val_accuracy: 0.9456
Epoch 7/10
465/465 [==============================] - 230s 494ms/step - loss: 0.0945 - accuracy: 0.9745 - val_loss: 0.166

In [39]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

17
Epoch 1/10
493/493 [==============================] - 247s 493ms/step - loss: 3.4610 - accuracy: 0.0834 - val_loss: 1.7534 - val_accuracy: 0.5925
Epoch 2/10
493/493 [==============================] - 244s 494ms/step - loss: 1.0035 - accuracy: 0.7181 - val_loss: 0.3677 - val_accuracy: 0.9255
Epoch 3/10
493/493 [==============================] - 244s 495ms/step - loss: 0.2479 - accuracy: 0.9269 - val_loss: 0.1735 - val_accuracy: 0.9467
Epoch 4/10
493/493 [==============================] - 243s 493ms/step - loss: 0.1430 - accuracy: 0.9605 - val_loss: 0.3232 - val_accuracy: 0.9456
Epoch 5/10
493/493 [==============================] - 245s 496ms/step - loss: 0.1044 - accuracy: 0.9703 - val_loss: 0.1769 - val_accuracy: 0.9492
Epoch 6/10
493/493 [==============================] - 270s 547ms/step - loss: 0.0948 - accuracy: 0.9718 - val_loss: 0.0946 - val_accuracy: 0.9691
Epoch 7/10
493/493 [==============================] - 243s 494ms/step - loss: 0.0797 - accuracy: 0.9794 - val_loss: 0.310

In [40]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

18
Epoch 1/10
520/520 [==============================] - 262s 495ms/step - loss: 3.3563 - accuracy: 0.1098 - val_loss: 0.8334 - val_accuracy: 0.7755
Epoch 2/10
520/520 [==============================] - 258s 495ms/step - loss: 0.6810 - accuracy: 0.8023 - val_loss: 0.1214 - val_accuracy: 0.9599
Epoch 3/10
520/520 [==============================] - 257s 495ms/step - loss: 0.2014 - accuracy: 0.9408 - val_loss: 0.1142 - val_accuracy: 0.9579
Epoch 4/10
520/520 [==============================] - 257s 494ms/step - loss: 0.1372 - accuracy: 0.9604 - val_loss: 0.0881 - val_accuracy: 0.9709
Epoch 5/10
520/520 [==============================] - 257s 495ms/step - loss: 0.1049 - accuracy: 0.9703 - val_loss: 0.0561 - val_accuracy: 0.9867
Epoch 6/10
520/520 [==============================] - 256s 491ms/step - loss: 0.0976 - accuracy: 0.9733 - val_loss: 0.0412 - val_accuracy: 0.9893
Epoch 7/10
520/520 [==============================] - 255s 490ms/step - loss: 0.0905 - accuracy: 0.9750 - val_loss: 0.033

In [41]:
print(n)#19
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

19
Epoch 1/10
547/547 [==============================] - 273s 492ms/step - loss: 3.3047 - accuracy: 0.1202 - val_loss: 0.5335 - val_accuracy: 0.8273
Epoch 2/10
547/547 [==============================] - 268s 491ms/step - loss: 0.6030 - accuracy: 0.8179 - val_loss: 0.0666 - val_accuracy: 0.9809
Epoch 3/10
547/547 [==============================] - 271s 495ms/step - loss: 0.1928 - accuracy: 0.9433 - val_loss: 0.0346 - val_accuracy: 0.9916
Epoch 4/10
547/547 [==============================] - 269s 493ms/step - loss: 0.1303 - accuracy: 0.9639 - val_loss: 0.0518 - val_accuracy: 0.9855
Epoch 5/10
547/547 [==============================] - 271s 495ms/step - loss: 0.1052 - accuracy: 0.9717 - val_loss: 0.0738 - val_accuracy: 0.9824
Epoch 6/10
547/547 [==============================] - 299s 548ms/step - loss: 0.0814 - accuracy: 0.9775 - val_loss: 0.0288 - val_accuracy: 0.9936
Epoch 7/10
547/547 [==============================] - 359s 657ms/step - loss: 0.0828 - accuracy: 0.9781 - val_loss: 0.025

In [46]:
print(n)#last
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

20
Epoch 1/10
552/552 [==============================] - 352s 620ms/step - loss: 3.4177 - accuracy: 0.0913 - val_loss: 1.4554 - val_accuracy: 0.5233
Epoch 2/10
552/552 [==============================] - 339s 614ms/step - loss: 1.0160 - accuracy: 0.6861 - val_loss: 0.1723 - val_accuracy: 0.9546
Epoch 3/10
552/552 [==============================] - 365s 661ms/step - loss: 0.2643 - accuracy: 0.9206 - val_loss: 0.0559 - val_accuracy: 0.9819
Epoch 4/10
552/552 [==============================] - 364s 659ms/step - loss: 0.1577 - accuracy: 0.9558 - val_loss: 0.0305 - val_accuracy: 0.9931
Epoch 5/10
552/552 [==============================] - 363s 658ms/step - loss: 0.1243 - accuracy: 0.9661 - val_loss: 0.1096 - val_accuracy: 0.9737
Epoch 6/10
552/552 [==============================] - 363s 657ms/step - loss: 0.0941 - accuracy: 0.9737 - val_loss: 0.0290 - val_accuracy: 0.9936
Epoch 7/10
552/552 [==============================] - 365s 661ms/step - loss: 0.0939 - accuracy: 0.9736 - val_loss: 0.030

In [ ]:
len(models_lsa)

In [ ]:
models_lsa[-1].evaluate(x_val,y_val)

In [43]:
len(models_lsa)

21

In [47]:
#new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
new_model_lsa_dir  = "D:/models/aug_22/gtsrb/C1-2/gtsrb_model_c1_aug_lsa_e1"

i=20

for model in models_lsa[-1:]:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1-2/gtsrb_model_c1_aug_lsa_e1_20\assets
Model has been saved


In [48]:
import gc

In [50]:
del image_sets_lsa
del label_sets_lsa
del lsa_values

NameError: name 'image_sets_lsa' is not defined

In [57]:
del models_lsa

In [58]:
gc.collect()

5416

In [ ]:
loading = False

models_lsa = []

if loading:
    for i in range(10,20):
        model_lsa_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_lsa_e1_"+str(i)
        print(model_lsa_dir)
        model =utils.My_model('gtsrb',True,model_lsa_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_lsa.append(model)
       
    

## Training guided by DSA

In [54]:
m = 1750
n = 0
image_sets_dsa = []
label_sets_dsa = []


for i in range((len(top_images_by_dsa)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dsa)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dsa)%m))
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m])
    image_sets_dsa.append(top_images_by_dsa_n)
    label_sets_dsa.append(top_labels_by_dsa_n)
    print(len(top_images_by_dsa_n))
    n += m



0 :
0  ->  1750
1750
1 :
0  ->  3500
3500
2 :
0  ->  5250
5250
3 :
0  ->  7000
7000
4 :
0  ->  8750
8750
5 :
0  ->  10500
10500
6 :
0  ->  12250
12250
7 :
0  ->  14000
14000
8 :
0  ->  15750
15750
9 :
0  ->  17500
17500
10 :
0  ->  19250
19250
11 :
0  ->  21000
21000
12 :
0  ->  22750
22750
13 :
0  ->  24500
24500
14 :
0  ->  26250
26250
15 :
0  ->  28000
28000
16 :
0  ->  29750
29750
17 :
0  ->  31500
31500
18 :
0  ->  33250
33250
19 :
0  ->  35000
35000
20 :
Last
0  ->  37037
35287


In [64]:
print(model_dir)

models_dsa = []
for i in range(len(label_sets_dsa)):
    print(i,":")
    model = utils.My_model('gtsrb',False,None)
    model.compile_model()
    models_dsa.append(model)


C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/
0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled
20 :
Model compiled


In [65]:
n=0
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

0
Epoch 1/10
28/28 [==============================] - 37s 978ms/step - loss: 3.9212 - accuracy: 0.0542 - val_loss: 3.5527 - val_accuracy: 0.0574
Epoch 2/10
28/28 [==============================] - 27s 980ms/step - loss: 3.5350 - accuracy: 0.0659 - val_loss: 3.5438 - val_accuracy: 0.0554
Epoch 3/10
28/28 [==============================] - 27s 969ms/step - loss: 3.5194 - accuracy: 0.0567 - val_loss: 3.4831 - val_accuracy: 0.0776
Epoch 4/10
28/28 [==============================] - 27s 972ms/step - loss: 3.4850 - accuracy: 0.0873 - val_loss: 3.4509 - val_accuracy: 0.0967
Epoch 5/10
28/28 [==============================] - 26s 956ms/step - loss: 3.4419 - accuracy: 0.0976 - val_loss: 3.4590 - val_accuracy: 0.0579
Epoch 6/10
28/28 [==============================] - 25s 900ms/step - loss: 3.4483 - accuracy: 0.0777 - val_loss: 3.4436 - val_accuracy: 0.0857
Epoch 7/10
28/28 [==============================] - 25s 917ms/step - loss: 3.4118 - accuracy: 0.0978 - val_loss: 3.4163 - val_accuracy: 0.07

In [66]:
n=1
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

1
Epoch 1/10
55/55 [==============================] - 59s 831ms/step - loss: 3.6809 - accuracy: 0.0522 - val_loss: 3.5126 - val_accuracy: 0.0498
Epoch 2/10
55/55 [==============================] - 44s 811ms/step - loss: 3.5639 - accuracy: 0.0632 - val_loss: 3.4273 - val_accuracy: 0.0873
Epoch 3/10
55/55 [==============================] - 44s 810ms/step - loss: 3.4784 - accuracy: 0.0883 - val_loss: 3.4011 - val_accuracy: 0.0908
Epoch 4/10
55/55 [==============================] - 44s 811ms/step - loss: 3.4525 - accuracy: 0.0913 - val_loss: 3.4166 - val_accuracy: 0.0875
Epoch 5/10
55/55 [==============================] - 46s 833ms/step - loss: 3.3746 - accuracy: 0.1080 - val_loss: 2.9297 - val_accuracy: 0.2085
Epoch 6/10
55/55 [==============================] - 45s 820ms/step - loss: 2.9960 - accuracy: 0.2009 - val_loss: 2.4161 - val_accuracy: 0.3174
Epoch 7/10
55/55 [==============================] - 44s 806ms/step - loss: 2.5526 - accuracy: 0.2871 - val_loss: 1.7408 - val_accuracy: 0.49

In [67]:
n=2
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

2
Epoch 1/10
83/83 [==============================] - 73s 764ms/step - loss: 3.5560 - accuracy: 0.1608 - val_loss: 3.6020 - val_accuracy: 0.0452
Epoch 2/10
83/83 [==============================] - 54s 656ms/step - loss: 3.1190 - accuracy: 0.2352 - val_loss: 3.5177 - val_accuracy: 0.0674
Epoch 3/10
83/83 [==============================] - 49s 586ms/step - loss: 2.7537 - accuracy: 0.3201 - val_loss: 3.3288 - val_accuracy: 0.1717
Epoch 4/10
83/83 [==============================] - 47s 564ms/step - loss: 2.1163 - accuracy: 0.4409 - val_loss: 2.5367 - val_accuracy: 0.2991
Epoch 5/10
83/83 [==============================] - 47s 564ms/step - loss: 1.7506 - accuracy: 0.5291 - val_loss: 1.4480 - val_accuracy: 0.6109
Epoch 6/10
83/83 [==============================] - 47s 562ms/step - loss: 1.3365 - accuracy: 0.6115 - val_loss: 1.4836 - val_accuracy: 0.5665
Epoch 7/10
83/83 [==============================] - 47s 567ms/step - loss: 1.0486 - accuracy: 0.7029 - val_loss: 0.6523 - val_accuracy: 0.80

In [68]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

3
Epoch 1/10
110/110 [==============================] - 64s 541ms/step - loss: 4.6983 - accuracy: 0.1639 - val_loss: 4.4671 - val_accuracy: 0.0513
Epoch 2/10
110/110 [==============================] - 58s 531ms/step - loss: 2.8858 - accuracy: 0.2582 - val_loss: 3.9251 - val_accuracy: 0.0643
Epoch 3/10
110/110 [==============================] - 59s 540ms/step - loss: 2.2965 - accuracy: 0.3652 - val_loss: 2.3294 - val_accuracy: 0.4075
Epoch 4/10
110/110 [==============================] - 59s 538ms/step - loss: 1.5056 - accuracy: 0.6096 - val_loss: 1.7552 - val_accuracy: 0.5022
Epoch 5/10
110/110 [==============================] - 80s 726ms/step - loss: 1.0796 - accuracy: 0.7082 - val_loss: 1.2883 - val_accuracy: 0.6652
Epoch 6/10
110/110 [==============================] - 79s 719ms/step - loss: 0.8268 - accuracy: 0.7558 - val_loss: 0.7558 - val_accuracy: 0.7880
Epoch 7/10
110/110 [==============================] - 80s 724ms/step - loss: 0.6274 - accuracy: 0.8172 - val_loss: 0.5614 - val_

In [69]:
print(n)#4

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

4
Epoch 1/10
137/137 [==============================] - 109s 729ms/step - loss: 3.2359 - accuracy: 0.1385 - val_loss: 3.6938 - val_accuracy: 0.0536
Epoch 2/10
137/137 [==============================] - 100s 731ms/step - loss: 2.7103 - accuracy: 0.2559 - val_loss: 2.9631 - val_accuracy: 0.2072
Epoch 3/10
137/137 [==============================] - 99s 720ms/step - loss: 1.7556 - accuracy: 0.5315 - val_loss: 2.3267 - val_accuracy: 0.3912
Epoch 4/10
137/137 [==============================] - 98s 713ms/step - loss: 1.1036 - accuracy: 0.7122 - val_loss: 1.3746 - val_accuracy: 0.6091
Epoch 5/10
137/137 [==============================] - 98s 712ms/step - loss: 0.7797 - accuracy: 0.7838 - val_loss: 0.9101 - val_accuracy: 0.7397
Epoch 6/10
137/137 [==============================] - 100s 732ms/step - loss: 0.5931 - accuracy: 0.8284 - val_loss: 0.6859 - val_accuracy: 0.8137
Epoch 7/10
137/137 [==============================] - 94s 683ms/step - loss: 0.4500 - accuracy: 0.8701 - val_loss: 0.4635 - v

In [70]:
n=5
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

5
Epoch 1/10
165/165 [==============================] - 103s 574ms/step - loss: 3.3693 - accuracy: 0.1445 - val_loss: 3.6543 - val_accuracy: 0.0607
Epoch 2/10
165/165 [==============================] - 94s 570ms/step - loss: 2.6638 - accuracy: 0.2139 - val_loss: 3.0212 - val_accuracy: 0.2268
Epoch 3/10
165/165 [==============================] - 2004s 12s/step - loss: 1.4455 - accuracy: 0.6414 - val_loss: 2.2014 - val_accuracy: 0.4407
Epoch 4/10
165/165 [==============================] - 107s 645ms/step - loss: 0.8816 - accuracy: 0.7759 - val_loss: 1.3187 - val_accuracy: 0.6124
Epoch 5/10
165/165 [==============================] - 86s 523ms/step - loss: 0.6353 - accuracy: 0.8223 - val_loss: 1.5069 - val_accuracy: 0.5596
Epoch 6/10
165/165 [==============================] - 86s 522ms/step - loss: 0.5063 - accuracy: 0.8538 - val_loss: 0.7367 - val_accuracy: 0.7997
Epoch 7/10
165/165 [==============================] - 90s 549ms/step - loss: 0.3893 - accuracy: 0.8830 - val_loss: 0.5129 - va

In [71]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

6
Epoch 1/10
192/192 [==============================] - 105s 520ms/step - loss: 3.1013 - accuracy: 0.1345 - val_loss: 3.6604 - val_accuracy: 0.0758
Epoch 2/10
192/192 [==============================] - 99s 514ms/step - loss: 2.4722 - accuracy: 0.2701 - val_loss: 2.5488 - val_accuracy: 0.3401
Epoch 3/10
192/192 [==============================] - 100s 522ms/step - loss: 1.0892 - accuracy: 0.7378 - val_loss: 1.6337 - val_accuracy: 0.5430
Epoch 4/10
192/192 [==============================] - 99s 517ms/step - loss: 0.6960 - accuracy: 0.8142 - val_loss: 1.1459 - val_accuracy: 0.6793
Epoch 5/10
192/192 [==============================] - 99s 516ms/step - loss: 0.4877 - accuracy: 0.8641 - val_loss: 0.7752 - val_accuracy: 0.7747
Epoch 6/10
192/192 [==============================] - 99s 517ms/step - loss: 0.3733 - accuracy: 0.8886 - val_loss: 0.6531 - val_accuracy: 0.8071
Epoch 7/10
192/192 [==============================] - 99s 515ms/step - loss: 0.2988 - accuracy: 0.9131 - val_loss: 0.3540 - va

In [72]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

7
Epoch 1/10
219/219 [==============================] - 119s 517ms/step - loss: 3.3514 - accuracy: 0.1375 - val_loss: 3.9724 - val_accuracy: 0.0875
Epoch 2/10
219/219 [==============================] - 113s 515ms/step - loss: 2.3080 - accuracy: 0.3289 - val_loss: 2.4471 - val_accuracy: 0.4024
Epoch 3/10
219/219 [==============================] - 114s 520ms/step - loss: 0.9292 - accuracy: 0.7723 - val_loss: 1.7215 - val_accuracy: 0.5662
Epoch 4/10
219/219 [==============================] - 112s 512ms/step - loss: 0.5536 - accuracy: 0.8598 - val_loss: 0.9658 - val_accuracy: 0.7714
Epoch 5/10
219/219 [==============================] - 113s 514ms/step - loss: 0.4197 - accuracy: 0.8864 - val_loss: 0.7080 - val_accuracy: 0.7948
Epoch 6/10
219/219 [==============================] - 113s 515ms/step - loss: 0.3164 - accuracy: 0.9128 - val_loss: 0.5067 - val_accuracy: 0.8464
Epoch 7/10
219/219 [==============================] - 112s 513ms/step - loss: 0.2708 - accuracy: 0.9215 - val_loss: 0.4016

In [73]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

8
Epoch 1/10
247/247 [==============================] - 130s 508ms/step - loss: 3.0004 - accuracy: 0.1414 - val_loss: 4.5777 - val_accuracy: 0.0763
Epoch 2/10
247/247 [==============================] - 127s 516ms/step - loss: 2.3467 - accuracy: 0.3105 - val_loss: 2.9735 - val_accuracy: 0.3981
Epoch 3/10
247/247 [==============================] - 125s 508ms/step - loss: 0.8873 - accuracy: 0.7688 - val_loss: 1.3511 - val_accuracy: 0.6535
Epoch 4/10
247/247 [==============================] - 125s 508ms/step - loss: 0.4962 - accuracy: 0.8692 - val_loss: 0.8920 - val_accuracy: 0.7777
Epoch 5/10
247/247 [==============================] - 125s 508ms/step - loss: 0.3629 - accuracy: 0.8996 - val_loss: 0.7127 - val_accuracy: 0.8398
Epoch 6/10
247/247 [==============================] - 126s 509ms/step - loss: 0.2767 - accuracy: 0.9256 - val_loss: 0.4521 - val_accuracy: 0.8882
Epoch 7/10
247/247 [==============================] - 127s 514ms/step - loss: 0.2367 - accuracy: 0.9340 - val_loss: 0.4594

In [74]:
print(n)#9

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

9
Epoch 1/10
274/274 [==============================] - 144s 512ms/step - loss: 3.4456 - accuracy: 0.1254 - val_loss: 3.5657 - val_accuracy: 0.0967
Epoch 2/10
274/274 [==============================] - 139s 506ms/step - loss: 2.2610 - accuracy: 0.3307 - val_loss: 2.1796 - val_accuracy: 0.4478
Epoch 3/10
274/274 [==============================] - 139s 508ms/step - loss: 0.7840 - accuracy: 0.7903 - val_loss: 1.3901 - val_accuracy: 0.6545
Epoch 4/10
274/274 [==============================] - 139s 506ms/step - loss: 0.4555 - accuracy: 0.8809 - val_loss: 1.0467 - val_accuracy: 0.7351
Epoch 5/10
274/274 [==============================] - 140s 510ms/step - loss: 0.3542 - accuracy: 0.9062 - val_loss: 0.5216 - val_accuracy: 0.8622
Epoch 6/10
274/274 [==============================] - 138s 505ms/step - loss: 0.2531 - accuracy: 0.9306 - val_loss: 0.4167 - val_accuracy: 0.8788
Epoch 7/10
274/274 [==============================] - 139s 506ms/step - loss: 0.2317 - accuracy: 0.9377 - val_loss: 0.3995

In [75]:
n=10
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

10
Epoch 1/10
301/301 [==============================] - 156s 505ms/step - loss: 2.9762 - accuracy: 0.1156 - val_loss: 3.6918 - val_accuracy: 0.1283
Epoch 2/10
301/301 [==============================] - 152s 507ms/step - loss: 2.0781 - accuracy: 0.3792 - val_loss: 2.0924 - val_accuracy: 0.5144
Epoch 3/10
301/301 [==============================] - 153s 507ms/step - loss: 0.6224 - accuracy: 0.8332 - val_loss: 1.2092 - val_accuracy: 0.7140
Epoch 4/10
301/301 [==============================] - 152s 507ms/step - loss: 0.3767 - accuracy: 0.9038 - val_loss: 0.7091 - val_accuracy: 0.8086
Epoch 5/10
301/301 [==============================] - 153s 507ms/step - loss: 0.2877 - accuracy: 0.9211 - val_loss: 0.5498 - val_accuracy: 0.8563
Epoch 6/10
301/301 [==============================] - 153s 507ms/step - loss: 0.2427 - accuracy: 0.9358 - val_loss: 0.4044 - val_accuracy: 0.8974
Epoch 7/10
301/301 [==============================] - 152s 506ms/step - loss: 0.2127 - accuracy: 0.9436 - val_loss: 0.330

In [76]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

11
Epoch 1/10
329/329 [==============================] - 170s 503ms/step - loss: 3.1804 - accuracy: 0.1051 - val_loss: 3.4592 - val_accuracy: 0.1978
Epoch 2/10
329/329 [==============================] - 167s 507ms/step - loss: 1.7667 - accuracy: 0.4830 - val_loss: 1.5490 - val_accuracy: 0.6818
Epoch 3/10
329/329 [==============================] - 166s 504ms/step - loss: 0.5070 - accuracy: 0.8730 - val_loss: 0.8145 - val_accuracy: 0.7910
Epoch 4/10
329/329 [==============================] - 167s 507ms/step - loss: 0.3069 - accuracy: 0.9154 - val_loss: 0.7262 - val_accuracy: 0.8168
Epoch 5/10
329/329 [==============================] - 166s 503ms/step - loss: 0.2405 - accuracy: 0.9357 - val_loss: 0.4229 - val_accuracy: 0.8859
Epoch 6/10
329/329 [==============================] - 166s 504ms/step - loss: 0.1944 - accuracy: 0.9425 - val_loss: 0.3586 - val_accuracy: 0.8972
Epoch 7/10
329/329 [==============================] - 166s 504ms/step - loss: 0.1772 - accuracy: 0.9497 - val_loss: 0.271

In [77]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

12
Epoch 1/10
356/356 [==============================] - 182s 500ms/step - loss: 3.2073 - accuracy: 0.0987 - val_loss: 3.0755 - val_accuracy: 0.2927
Epoch 2/10
356/356 [==============================] - 179s 503ms/step - loss: 1.5694 - accuracy: 0.5466 - val_loss: 1.5317 - val_accuracy: 0.6619
Epoch 3/10
356/356 [==============================] - 180s 507ms/step - loss: 0.4801 - accuracy: 0.8743 - val_loss: 0.6600 - val_accuracy: 0.8188
Epoch 4/10
356/356 [==============================] - 178s 501ms/step - loss: 0.2902 - accuracy: 0.9230 - val_loss: 0.3998 - val_accuracy: 0.8734
Epoch 5/10
356/356 [==============================] - 178s 501ms/step - loss: 0.2313 - accuracy: 0.9349 - val_loss: 0.5182 - val_accuracy: 0.8933
Epoch 6/10
356/356 [==============================] - 178s 501ms/step - loss: 0.1835 - accuracy: 0.9494 - val_loss: 0.2400 - val_accuracy: 0.9380
Epoch 7/10
356/356 [==============================] - 178s 500ms/step - loss: 0.1650 - accuracy: 0.9548 - val_loss: 0.293

In [78]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

13
Epoch 1/10
383/383 [==============================] - 196s 499ms/step - loss: 3.2296 - accuracy: 0.1031 - val_loss: 3.2238 - val_accuracy: 0.3776
Epoch 2/10
383/383 [==============================] - 192s 502ms/step - loss: 1.2810 - accuracy: 0.6330 - val_loss: 0.9638 - val_accuracy: 0.7857
Epoch 3/10
383/383 [==============================] - 193s 504ms/step - loss: 0.3885 - accuracy: 0.8990 - val_loss: 0.5995 - val_accuracy: 0.8548
Epoch 4/10
383/383 [==============================] - 192s 501ms/step - loss: 0.2287 - accuracy: 0.9388 - val_loss: 0.3464 - val_accuracy: 0.8956
Epoch 5/10
383/383 [==============================] - 192s 500ms/step - loss: 0.1718 - accuracy: 0.9537 - val_loss: 0.1692 - val_accuracy: 0.9502
Epoch 6/10
383/383 [==============================] - 191s 500ms/step - loss: 0.1501 - accuracy: 0.9568 - val_loss: 0.1427 - val_accuracy: 0.9650
Epoch 7/10
383/383 [==============================] - 192s 501ms/step - loss: 0.1288 - accuracy: 0.9634 - val_loss: 0.147

In [79]:
print(n)#14

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

14
Epoch 1/10
411/411 [==============================] - 208s 497ms/step - loss: 3.2841 - accuracy: 0.1117 - val_loss: 2.5737 - val_accuracy: 0.4889
Epoch 2/10
411/411 [==============================] - 206s 501ms/step - loss: 1.0517 - accuracy: 0.7073 - val_loss: 0.6961 - val_accuracy: 0.8515
Epoch 3/10
411/411 [==============================] - 205s 498ms/step - loss: 0.3061 - accuracy: 0.9183 - val_loss: 0.5379 - val_accuracy: 0.8778
Epoch 4/10
411/411 [==============================] - 205s 498ms/step - loss: 0.1894 - accuracy: 0.9474 - val_loss: 0.1663 - val_accuracy: 0.9561
Epoch 5/10
411/411 [==============================] - 204s 497ms/step - loss: 0.1397 - accuracy: 0.9618 - val_loss: 0.1159 - val_accuracy: 0.9696
Epoch 6/10
411/411 [==============================] - 204s 497ms/step - loss: 0.1320 - accuracy: 0.9632 - val_loss: 0.1358 - val_accuracy: 0.9709
Epoch 7/10
411/411 [==============================] - 212s 516ms/step - loss: 0.1155 - accuracy: 0.9701 - val_loss: 0.077

In [80]:
n=15
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

15
Epoch 1/10
438/438 [==============================] - 223s 499ms/step - loss: 3.3826 - accuracy: 0.0873 - val_loss: 2.2732 - val_accuracy: 0.4601
Epoch 2/10
438/438 [==============================] - 218s 497ms/step - loss: 1.3257 - accuracy: 0.6216 - val_loss: 0.7905 - val_accuracy: 0.8548
Epoch 3/10
438/438 [==============================] - 218s 497ms/step - loss: 0.3819 - accuracy: 0.8960 - val_loss: 0.4906 - val_accuracy: 0.8893
Epoch 4/10
438/438 [==============================] - 217s 496ms/step - loss: 0.2208 - accuracy: 0.9390 - val_loss: 0.2874 - val_accuracy: 0.9286
Epoch 5/10
438/438 [==============================] - 217s 496ms/step - loss: 0.1845 - accuracy: 0.9516 - val_loss: 0.1924 - val_accuracy: 0.9528
Epoch 6/10
438/438 [==============================] - 217s 496ms/step - loss: 0.1513 - accuracy: 0.9605 - val_loss: 0.1237 - val_accuracy: 0.9678
Epoch 7/10
438/438 [==============================] - 217s 495ms/step - loss: 0.1245 - accuracy: 0.9636 - val_loss: 0.153

In [81]:
import time
time.sleep(400)

In [82]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

16
Epoch 1/10
465/465 [==============================] - 205s 432ms/step - loss: 3.3544 - accuracy: 0.0953 - val_loss: 1.6787 - val_accuracy: 0.6137
Epoch 2/10
465/465 [==============================] - 212s 457ms/step - loss: 1.0253 - accuracy: 0.7032 - val_loss: 0.4901 - val_accuracy: 0.8803
Epoch 3/10
465/465 [==============================] - 223s 480ms/step - loss: 0.3169 - accuracy: 0.9135 - val_loss: 0.3614 - val_accuracy: 0.9071
Epoch 4/10
465/465 [==============================] - 230s 495ms/step - loss: 0.1881 - accuracy: 0.9480 - val_loss: 0.2083 - val_accuracy: 0.9316
Epoch 5/10
465/465 [==============================] - 231s 496ms/step - loss: 0.1504 - accuracy: 0.9609 - val_loss: 0.1966 - val_accuracy: 0.9497
Epoch 6/10
465/465 [==============================] - 230s 495ms/step - loss: 0.1276 - accuracy: 0.9661 - val_loss: 0.1040 - val_accuracy: 0.9699
Epoch 7/10
465/465 [==============================] - 231s 496ms/step - loss: 0.1128 - accuracy: 0.9691 - val_loss: 0.091

In [83]:

time.sleep(400)

In [84]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

17
Epoch 1/10
493/493 [==============================] - 215s 427ms/step - loss: 3.3743 - accuracy: 0.0903 - val_loss: 1.7142 - val_accuracy: 0.5343
Epoch 2/10
493/493 [==============================] - 215s 435ms/step - loss: 0.9804 - accuracy: 0.7113 - val_loss: 0.5072 - val_accuracy: 0.8905
Epoch 3/10
493/493 [==============================] - 222s 451ms/step - loss: 0.2663 - accuracy: 0.9274 - val_loss: 0.2957 - val_accuracy: 0.9393
Epoch 4/10
493/493 [==============================] - 237s 480ms/step - loss: 0.1595 - accuracy: 0.9564 - val_loss: 0.1684 - val_accuracy: 0.9515
Epoch 5/10
493/493 [==============================] - 243s 493ms/step - loss: 0.1270 - accuracy: 0.9660 - val_loss: 0.2573 - val_accuracy: 0.9423
Epoch 6/10
493/493 [==============================] - 245s 498ms/step - loss: 0.1145 - accuracy: 0.9673 - val_loss: 0.0942 - val_accuracy: 0.9678
Epoch 7/10
493/493 [==============================] - 243s 492ms/step - loss: 0.1101 - accuracy: 0.9727 - val_loss: 0.076

In [85]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

18
Epoch 1/10
520/520 [==============================] - 261s 493ms/step - loss: 3.3032 - accuracy: 0.1070 - val_loss: 1.2044 - val_accuracy: 0.6844
Epoch 2/10
520/520 [==============================] - 257s 495ms/step - loss: 0.8724 - accuracy: 0.7415 - val_loss: 0.3868 - val_accuracy: 0.8969
Epoch 3/10
520/520 [==============================] - 258s 496ms/step - loss: 0.2546 - accuracy: 0.9279 - val_loss: 0.2186 - val_accuracy: 0.9520
Epoch 4/10
520/520 [==============================] - 257s 494ms/step - loss: 0.1563 - accuracy: 0.9552 - val_loss: 0.1045 - val_accuracy: 0.9717
Epoch 5/10
520/520 [==============================] - 256s 493ms/step - loss: 0.1115 - accuracy: 0.9694 - val_loss: 0.1341 - val_accuracy: 0.9673
Epoch 6/10
520/520 [==============================] - 258s 496ms/step - loss: 0.0983 - accuracy: 0.9735 - val_loss: 0.0873 - val_accuracy: 0.9826
Epoch 7/10
520/520 [==============================] - 257s 494ms/step - loss: 0.1012 - accuracy: 0.9728 - val_loss: 0.085

In [86]:
n=19
print(n)#19

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

19
Epoch 1/10
547/547 [==============================] - 284s 510ms/step - loss: 3.2615 - accuracy: 0.1323 - val_loss: 0.4813 - val_accuracy: 0.8576
Epoch 2/10
547/547 [==============================] - 270s 494ms/step - loss: 0.5925 - accuracy: 0.8227 - val_loss: 0.1154 - val_accuracy: 0.9786
Epoch 3/10
547/547 [==============================] - 271s 495ms/step - loss: 0.2003 - accuracy: 0.9404 - val_loss: 0.0859 - val_accuracy: 0.9867
Epoch 4/10
547/547 [==============================] - 270s 494ms/step - loss: 0.1293 - accuracy: 0.9626 - val_loss: 0.0539 - val_accuracy: 0.9872
Epoch 5/10
547/547 [==============================] - 270s 493ms/step - loss: 0.0995 - accuracy: 0.9726 - val_loss: 0.0339 - val_accuracy: 0.9921
Epoch 6/10
547/547 [==============================] - 269s 491ms/step - loss: 0.0871 - accuracy: 0.9754 - val_loss: 0.0295 - val_accuracy: 0.9946
Epoch 7/10
547/547 [==============================] - 269s 492ms/step - loss: 0.0828 - accuracy: 0.9771 - val_loss: 0.032

In [87]:
n=20
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

20
Epoch 1/10
552/552 [==============================] - 276s 492ms/step - loss: 3.3701 - accuracy: 0.1060 - val_loss: 0.7863 - val_accuracy: 0.7517
Epoch 2/10
552/552 [==============================] - 273s 495ms/step - loss: 0.7010 - accuracy: 0.7893 - val_loss: 0.1102 - val_accuracy: 0.9701
Epoch 3/10
552/552 [==============================] - 272s 493ms/step - loss: 0.2027 - accuracy: 0.9425 - val_loss: 0.0267 - val_accuracy: 0.9923
Epoch 4/10
552/552 [==============================] - 275s 497ms/step - loss: 0.1266 - accuracy: 0.9627 - val_loss: 0.0340 - val_accuracy: 0.9936
Epoch 5/10
552/552 [==============================] - 272s 493ms/step - loss: 0.1087 - accuracy: 0.9686 - val_loss: 0.0227 - val_accuracy: 0.9957
Epoch 6/10
552/552 [==============================] - 272s 493ms/step - loss: 0.0932 - accuracy: 0.9738 - val_loss: 0.0394 - val_accuracy: 0.9936
Epoch 7/10
552/552 [==============================] - 274s 496ms/step - loss: 0.0851 - accuracy: 0.9772 - val_loss: 0.023

In [89]:
new_model_dsa_dir  = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dsa_e1"

i=0

for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dsa_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dsa_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dsa_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dsa_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dsa_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dsa_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dsa_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dsa_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gts

In [ ]:
loading = True

models_dsa = []

if loading:
    for i in range(20):
        model_dsa_dir = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2_"+str(i)
        print(model_dsa_dir)
        model =utils.My_model('gtsrb',True,model_dsa_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_dsa.append(model)
       
    

In [ ]:
try:
    del dsa_values
    del top_images_by_dsa
    del top_labels_by_dsa
    del image_sets_dsa
    del label_sets_dsa
    del models_dsa
except:
    print("Error")

In [ ]:
gc.collect()

## Uncertainty

In [15]:
dg_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/deep_gini_values.npy"

deep_gini_values = np.load(dg_direction)


In [16]:
# Obtaining top n images by dg values
top_images_by_dg  = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),x_train_and_adversary)
top_labels_by_dg = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),y_train_and_adversary)

In [17]:
m = n_data_points
n = 0
image_sets_dg = []
label_sets_dg = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_dg)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dg)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dg)%m))
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m+(len(top_images_by_dg)%m)])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m+(len(top_images_by_dg)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m])
    image_sets_dg.append(top_images_by_dg_n)
    label_sets_dg.append(top_labels_by_dg_n)
    print(len(top_images_by_dg_n))
    n += m



0 :
0  ->  1750
1750
1 :
0  ->  3500
3500
2 :
0  ->  5250
5250
3 :
0  ->  7000
7000
4 :
0  ->  8750
8750
5 :
0  ->  10500
10500
6 :
0  ->  12250
12250
7 :
0  ->  14000
14000
8 :
0  ->  15750
15750
9 :
0  ->  17500
17500
10 :
0  ->  19250
19250
11 :
0  ->  21000
21000
12 :
0  ->  22750
22750
13 :
0  ->  24500
24500
14 :
0  ->  26250
26250
15 :
0  ->  28000
28000
16 :
0  ->  29750
29750
17 :
0  ->  31500
31500
18 :
0  ->  33250
33250
19 :
0  ->  35000
35000
20 :
Last
0  ->  37037
35287


In [18]:
models_dg = []
#for i in range(len(label_sets_lsa)):
for i in range(len(label_sets_dg)):
    print(i,":")
    model = utils.My_model(dataset,False)
    model.compile_model()
    models_dg.append(model)

0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled
20 :
Model compiled


In [ ]:
n=0
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

0
Epoch 1/10
28/28 [==============================] - 96s 933ms/step - loss: 3.8220 - accuracy: 0.0745 - val_loss: 3.5440 - val_accuracy: 0.0566
Epoch 2/10
28/28 [==============================] - 23s 816ms/step - loss: 3.4775 - accuracy: 0.0902 - val_loss: 3.5465 - val_accuracy: 0.0566
Epoch 3/10
28/28 [==============================] - 21s 761ms/step - loss: 3.4510 - accuracy: 0.1146 - val_loss: 3.5762 - val_accuracy: 0.0566
Epoch 4/10
28/28 [==============================] - 22s 794ms/step - loss: 3.4835 - accuracy: 0.1354 - val_loss: 3.5484 - val_accuracy: 0.0566
Epoch 5/10
28/28 [==============================] - 24s 852ms/step - loss: 3.4325 - accuracy: 0.1071 - val_loss: 3.5512 - val_accuracy: 0.0566
Epoch 6/10
28/28 [==============================] - 25s 916ms/step - loss: 3.4109 - accuracy: 0.1244 - val_loss: 3.5280 - val_accuracy: 0.0566
Epoch 7/10
28/28 [==============================] - 26s 950ms/step - loss: 3.3830 - accuracy: 0.1130 - val_loss: 3.5673 - val_accuracy: 0.05

In [34]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

1
Epoch 1/10
55/55 [==============================] - 35s 566ms/step - loss: 3.6983 - accuracy: 0.0946 - val_loss: 3.6527 - val_accuracy: 0.0566
Epoch 2/10
55/55 [==============================] - 31s 566ms/step - loss: 3.3817 - accuracy: 0.1342 - val_loss: 3.5867 - val_accuracy: 0.0748
Epoch 3/10
55/55 [==============================] - 31s 568ms/step - loss: 3.3353 - accuracy: 0.1580 - val_loss: 3.5494 - val_accuracy: 0.0758
Epoch 4/10
55/55 [==============================] - 31s 570ms/step - loss: 3.2806 - accuracy: 0.1575 - val_loss: 3.5513 - val_accuracy: 0.0786
Epoch 5/10
55/55 [==============================] - 32s 581ms/step - loss: 3.2397 - accuracy: 0.1726 - val_loss: 3.6669 - val_accuracy: 0.0750
Epoch 6/10
55/55 [==============================] - 33s 604ms/step - loss: 3.1836 - accuracy: 0.1736 - val_loss: 3.5021 - val_accuracy: 0.0852
Epoch 7/10
55/55 [==============================] - 34s 614ms/step - loss: 3.0716 - accuracy: 0.1943 - val_loss: 3.4625 - val_accuracy: 0.09

In [35]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

2
Epoch 1/10
83/83 [==============================] - 51s 559ms/step - loss: 3.5855 - accuracy: 0.0876 - val_loss: 3.6179 - val_accuracy: 0.0722
Epoch 2/10
83/83 [==============================] - 46s 553ms/step - loss: 3.3701 - accuracy: 0.1351 - val_loss: 4.0322 - val_accuracy: 0.0482
Epoch 3/10
83/83 [==============================] - 46s 554ms/step - loss: 3.3427 - accuracy: 0.1537 - val_loss: 3.5803 - val_accuracy: 0.0857
Epoch 4/10
83/83 [==============================] - 46s 554ms/step - loss: 3.1204 - accuracy: 0.1837 - val_loss: 3.0674 - val_accuracy: 0.1837
Epoch 5/10
83/83 [==============================] - 46s 558ms/step - loss: 2.7281 - accuracy: 0.2464 - val_loss: 2.8838 - val_accuracy: 0.2376
Epoch 6/10
83/83 [==============================] - 46s 556ms/step - loss: 2.2617 - accuracy: 0.3609 - val_loss: 2.3584 - val_accuracy: 0.4394
Epoch 7/10
83/83 [==============================] - 46s 560ms/step - loss: 1.8735 - accuracy: 0.4676 - val_loss: 1.7355 - val_accuracy: 0.52

In [43]:
n=3
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

3
Epoch 1/10
110/110 [==============================] - 84s 675ms/step - loss: 3.5843 - accuracy: 0.0904 - val_loss: 3.6086 - val_accuracy: 0.0740
Epoch 2/10
110/110 [==============================] - 58s 527ms/step - loss: 3.3519 - accuracy: 0.1487 - val_loss: 3.6831 - val_accuracy: 0.0766
Epoch 3/10
110/110 [==============================] - 59s 538ms/step - loss: 3.2293 - accuracy: 0.1687 - val_loss: 3.2191 - val_accuracy: 0.1363
Epoch 4/10
110/110 [==============================] - 61s 551ms/step - loss: 2.7776 - accuracy: 0.2417 - val_loss: 2.4888 - val_accuracy: 0.2728
Epoch 5/10
110/110 [==============================] - 61s 554ms/step - loss: 2.2279 - accuracy: 0.3506 - val_loss: 1.6213 - val_accuracy: 0.5509
Epoch 6/10
110/110 [==============================] - 62s 560ms/step - loss: 1.6199 - accuracy: 0.5261 - val_loss: 1.3148 - val_accuracy: 0.6440
Epoch 7/10
110/110 [==============================] - 61s 552ms/step - loss: 1.2360 - accuracy: 0.6295 - val_loss: 1.0339 - val_

In [44]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

4
Epoch 1/10
137/137 [==============================] - 79s 544ms/step - loss: 3.5679 - accuracy: 0.0874 - val_loss: 3.5966 - val_accuracy: 0.0755
Epoch 2/10
137/137 [==============================] - 74s 538ms/step - loss: 3.3493 - accuracy: 0.1565 - val_loss: 3.6995 - val_accuracy: 0.0806
Epoch 3/10
137/137 [==============================] - 76s 552ms/step - loss: 2.9843 - accuracy: 0.1910 - val_loss: 2.6994 - val_accuracy: 0.2888
Epoch 4/10
137/137 [==============================] - 75s 550ms/step - loss: 2.2544 - accuracy: 0.3595 - val_loss: 1.5462 - val_accuracy: 0.5729
Epoch 5/10
137/137 [==============================] - 75s 545ms/step - loss: 1.4971 - accuracy: 0.5614 - val_loss: 0.8582 - val_accuracy: 0.7328
Epoch 6/10
137/137 [==============================] - 75s 545ms/step - loss: 1.0618 - accuracy: 0.6743 - val_loss: 0.5895 - val_accuracy: 0.8181
Epoch 7/10
137/137 [==============================] - 75s 545ms/step - loss: 0.7469 - accuracy: 0.7690 - val_loss: 0.4668 - val_

In [45]:
print(n)#5
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

5
Epoch 1/10
165/165 [==============================] - 93s 540ms/step - loss: 3.6692 - accuracy: 0.0802 - val_loss: 3.5803 - val_accuracy: 0.0842
Epoch 2/10
165/165 [==============================] - 87s 531ms/step - loss: 3.2964 - accuracy: 0.1426 - val_loss: 3.4726 - val_accuracy: 0.1110
Epoch 3/10
165/165 [==============================] - 88s 533ms/step - loss: 2.7914 - accuracy: 0.2164 - val_loss: 2.5776 - val_accuracy: 0.2689
Epoch 4/10
165/165 [==============================] - 89s 537ms/step - loss: 1.9151 - accuracy: 0.4367 - val_loss: 1.3816 - val_accuracy: 0.6165
Epoch 5/10
165/165 [==============================] - 88s 535ms/step - loss: 1.1708 - accuracy: 0.6482 - val_loss: 0.8820 - val_accuracy: 0.7265
Epoch 6/10
165/165 [==============================] - 89s 541ms/step - loss: 0.7891 - accuracy: 0.7545 - val_loss: 0.3265 - val_accuracy: 0.8798
Epoch 7/10
165/165 [==============================] - 99s 599ms/step - loss: 0.5255 - accuracy: 0.8355 - val_loss: 0.2428 - val_

In [46]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

6
Epoch 1/10
192/192 [==============================] - 108s 536ms/step - loss: 3.4394 - accuracy: 0.0944 - val_loss: 3.6372 - val_accuracy: 0.0773
Epoch 2/10
192/192 [==============================] - 102s 532ms/step - loss: 2.7650 - accuracy: 0.2572 - val_loss: 2.4164 - val_accuracy: 0.3881
Epoch 3/10
192/192 [==============================] - 102s 533ms/step - loss: 1.5447 - accuracy: 0.5669 - val_loss: 0.9636 - val_accuracy: 0.7617
Epoch 4/10
192/192 [==============================] - 103s 536ms/step - loss: 0.8989 - accuracy: 0.7356 - val_loss: 0.5182 - val_accuracy: 0.8497
Epoch 5/10
192/192 [==============================] - 102s 532ms/step - loss: 0.5842 - accuracy: 0.8211 - val_loss: 0.2329 - val_accuracy: 0.9293
Epoch 6/10
192/192 [==============================] - 102s 531ms/step - loss: 0.4318 - accuracy: 0.8727 - val_loss: 0.1481 - val_accuracy: 0.9538
Epoch 7/10
192/192 [==============================] - 102s 530ms/step - loss: 0.3182 - accuracy: 0.9041 - val_loss: 0.1611

In [47]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

7
Epoch 1/10
219/219 [==============================] - 118s 521ms/step - loss: 3.3508 - accuracy: 0.1054 - val_loss: 3.5159 - val_accuracy: 0.0916
Epoch 2/10
219/219 [==============================] - 114s 522ms/step - loss: 2.5996 - accuracy: 0.2976 - val_loss: 2.0800 - val_accuracy: 0.4537
Epoch 3/10
219/219 [==============================] - 114s 521ms/step - loss: 1.3026 - accuracy: 0.6293 - val_loss: 1.0995 - val_accuracy: 0.7244
Epoch 4/10
219/219 [==============================] - 114s 522ms/step - loss: 0.7175 - accuracy: 0.7866 - val_loss: 0.3057 - val_accuracy: 0.9122
Epoch 5/10
219/219 [==============================] - 114s 522ms/step - loss: 0.4418 - accuracy: 0.8646 - val_loss: 0.1505 - val_accuracy: 0.9556
Epoch 6/10
219/219 [==============================] - 114s 521ms/step - loss: 0.3086 - accuracy: 0.9048 - val_loss: 0.1181 - val_accuracy: 0.9673
Epoch 7/10
219/219 [==============================] - 114s 522ms/step - loss: 0.2476 - accuracy: 0.9237 - val_loss: 0.0795

In [48]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

8
Epoch 1/10
247/247 [==============================] - 122s 478ms/step - loss: 3.4353 - accuracy: 0.1120 - val_loss: 3.7373 - val_accuracy: 0.1357
Epoch 2/10
247/247 [==============================] - 125s 507ms/step - loss: 2.3642 - accuracy: 0.3554 - val_loss: 1.9500 - val_accuracy: 0.4264
Epoch 3/10
247/247 [==============================] - 128s 517ms/step - loss: 1.0849 - accuracy: 0.6953 - val_loss: 0.7932 - val_accuracy: 0.7441
Epoch 4/10
247/247 [==============================] - 128s 518ms/step - loss: 0.6375 - accuracy: 0.8075 - val_loss: 0.2927 - val_accuracy: 0.9173
Epoch 5/10
247/247 [==============================] - 127s 515ms/step - loss: 0.4064 - accuracy: 0.8780 - val_loss: 0.1507 - val_accuracy: 0.9518
Epoch 6/10
247/247 [==============================] - 128s 517ms/step - loss: 0.2849 - accuracy: 0.9110 - val_loss: 0.0721 - val_accuracy: 0.9781
Epoch 7/10
247/247 [==============================] - 129s 522ms/step - loss: 0.2325 - accuracy: 0.9335 - val_loss: 0.0651

In [49]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

9
Epoch 1/10
274/274 [==============================] - 146s 515ms/step - loss: 3.2849 - accuracy: 0.1245 - val_loss: 3.3532 - val_accuracy: 0.1643
Epoch 2/10
274/274 [==============================] - 140s 510ms/step - loss: 2.1505 - accuracy: 0.4008 - val_loss: 1.6791 - val_accuracy: 0.5343
Epoch 3/10
274/274 [==============================] - 140s 510ms/step - loss: 0.9894 - accuracy: 0.7142 - val_loss: 0.8226 - val_accuracy: 0.8242
Epoch 4/10
274/274 [==============================] - 140s 510ms/step - loss: 0.5502 - accuracy: 0.8418 - val_loss: 0.1797 - val_accuracy: 0.9444
Epoch 5/10
274/274 [==============================] - 181s 660ms/step - loss: 0.3302 - accuracy: 0.8997 - val_loss: 0.0882 - val_accuracy: 0.9719
Epoch 6/10
274/274 [==============================] - 183s 667ms/step - loss: 0.2457 - accuracy: 0.9248 - val_loss: 0.0671 - val_accuracy: 0.9819
Epoch 7/10
274/274 [==============================] - 184s 671ms/step - loss: 0.1870 - accuracy: 0.9440 - val_loss: 0.0508

In [50]:
print(n)#10
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

10
Epoch 1/10
301/301 [==============================] - 194s 606ms/step - loss: 3.2590 - accuracy: 0.1173 - val_loss: 3.2832 - val_accuracy: 0.1878
Epoch 2/10
301/301 [==============================] - 198s 659ms/step - loss: 1.8997 - accuracy: 0.4776 - val_loss: 1.0843 - val_accuracy: 0.7571
Epoch 3/10
301/301 [==============================] - 199s 660ms/step - loss: 0.7800 - accuracy: 0.7808 - val_loss: 0.4979 - val_accuracy: 0.8785
Epoch 4/10
301/301 [==============================] - 192s 638ms/step - loss: 0.4620 - accuracy: 0.8617 - val_loss: 0.1841 - val_accuracy: 0.9431
Epoch 5/10
301/301 [==============================] - 202s 671ms/step - loss: 0.3190 - accuracy: 0.9025 - val_loss: 0.1211 - val_accuracy: 0.9684
Epoch 6/10
301/301 [==============================] - 202s 671ms/step - loss: 0.2469 - accuracy: 0.9259 - val_loss: 0.0707 - val_accuracy: 0.9788
Epoch 7/10
301/301 [==============================] - 202s 671ms/step - loss: 0.1962 - accuracy: 0.9404 - val_loss: 0.051

In [19]:
n=11
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

11
Epoch 1/10
329/329 [==============================] - 215s 448ms/step - loss: 3.2790 - accuracy: 0.1049 - val_loss: 3.4912 - val_accuracy: 0.1505
Epoch 2/10
329/329 [==============================] - 128s 389ms/step - loss: 2.1339 - accuracy: 0.3664 - val_loss: 1.2319 - val_accuracy: 0.7140
Epoch 3/10
329/329 [==============================] - 129s 392ms/step - loss: 0.7601 - accuracy: 0.7816 - val_loss: 0.6166 - val_accuracy: 0.8359
Epoch 4/10
329/329 [==============================] - 130s 394ms/step - loss: 0.3984 - accuracy: 0.8804 - val_loss: 0.1440 - val_accuracy: 0.9561
Epoch 5/10
329/329 [==============================] - 131s 400ms/step - loss: 0.2626 - accuracy: 0.9215 - val_loss: 0.0782 - val_accuracy: 0.9778
Epoch 6/10
329/329 [==============================] - 140s 424ms/step - loss: 0.1905 - accuracy: 0.9454 - val_loss: 0.0370 - val_accuracy: 0.9906
Epoch 7/10
329/329 [==============================] - 146s 445ms/step - loss: 0.1666 - accuracy: 0.9537 - val_loss: 0.128

In [20]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

12
Epoch 1/10
356/356 [==============================] - 163s 448ms/step - loss: 3.3040 - accuracy: 0.1038 - val_loss: 2.7618 - val_accuracy: 0.2557
Epoch 2/10
356/356 [==============================] - 172s 484ms/step - loss: 1.7343 - accuracy: 0.4932 - val_loss: 0.6569 - val_accuracy: 0.8352
Epoch 3/10
356/356 [==============================] - 169s 475ms/step - loss: 0.5674 - accuracy: 0.8374 - val_loss: 0.1982 - val_accuracy: 0.9490
Epoch 4/10
356/356 [==============================] - 168s 472ms/step - loss: 0.3144 - accuracy: 0.9071 - val_loss: 0.1080 - val_accuracy: 0.9640
Epoch 5/10
356/356 [==============================] - 169s 475ms/step - loss: 0.2146 - accuracy: 0.9370 - val_loss: 0.0590 - val_accuracy: 0.9849
Epoch 6/10
356/356 [==============================] - 171s 480ms/step - loss: 0.1730 - accuracy: 0.9505 - val_loss: 0.0747 - val_accuracy: 0.9786
Epoch 7/10
356/356 [==============================] - 169s 475ms/step - loss: 0.1407 - accuracy: 0.9596 - val_loss: 0.033

In [21]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

13
Epoch 1/10
383/383 [==============================] - 188s 478ms/step - loss: 3.4779 - accuracy: 0.0943 - val_loss: 2.7511 - val_accuracy: 0.2774
Epoch 2/10
383/383 [==============================] - 182s 476ms/step - loss: 1.6315 - accuracy: 0.5257 - val_loss: 0.5643 - val_accuracy: 0.8540
Epoch 3/10
383/383 [==============================] - 188s 492ms/step - loss: 0.4782 - accuracy: 0.8577 - val_loss: 0.1379 - val_accuracy: 0.9589
Epoch 4/10
383/383 [==============================] - 187s 487ms/step - loss: 0.2499 - accuracy: 0.9264 - val_loss: 0.0843 - val_accuracy: 0.9770
Epoch 5/10
383/383 [==============================] - 182s 476ms/step - loss: 0.1864 - accuracy: 0.9490 - val_loss: 0.0319 - val_accuracy: 0.9903
Epoch 6/10
383/383 [==============================] - 185s 483ms/step - loss: 0.1365 - accuracy: 0.9602 - val_loss: 0.0640 - val_accuracy: 0.9849
Epoch 7/10
383/383 [==============================] - 182s 476ms/step - loss: 0.1197 - accuracy: 0.9662 - val_loss: 0.033

In [22]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

14
Epoch 1/10
411/411 [==============================] - 185s 442ms/step - loss: 3.3776 - accuracy: 0.0935 - val_loss: 2.3617 - val_accuracy: 0.3674
Epoch 2/10
411/411 [==============================] - 177s 432ms/step - loss: 1.3979 - accuracy: 0.5944 - val_loss: 0.3310 - val_accuracy: 0.9127
Epoch 3/10
411/411 [==============================] - 176s 428ms/step - loss: 0.3960 - accuracy: 0.8855 - val_loss: 0.1112 - val_accuracy: 0.9630
Epoch 4/10
411/411 [==============================] - 178s 432ms/step - loss: 0.2098 - accuracy: 0.9400 - val_loss: 0.0989 - val_accuracy: 0.9689
Epoch 5/10
411/411 [==============================] - 178s 434ms/step - loss: 0.1539 - accuracy: 0.9557 - val_loss: 0.0572 - val_accuracy: 0.9809
Epoch 6/10
411/411 [==============================] - 184s 447ms/step - loss: 0.1248 - accuracy: 0.9642 - val_loss: 0.0279 - val_accuracy: 0.9908
Epoch 7/10
411/411 [==============================] - 198s 483ms/step - loss: 0.1083 - accuracy: 0.9692 - val_loss: 0.027

In [23]:
print(n)#15
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

15
Epoch 1/10
438/438 [==============================] - 189s 425ms/step - loss: 3.2917 - accuracy: 0.1172 - val_loss: 1.4826 - val_accuracy: 0.6331
Epoch 2/10
438/438 [==============================] - 187s 427ms/step - loss: 0.9289 - accuracy: 0.7355 - val_loss: 0.2751 - val_accuracy: 0.9178
Epoch 3/10
438/438 [==============================] - 188s 428ms/step - loss: 0.3055 - accuracy: 0.9115 - val_loss: 0.0778 - val_accuracy: 0.9786
Epoch 4/10
438/438 [==============================] - 187s 428ms/step - loss: 0.1754 - accuracy: 0.9505 - val_loss: 0.0681 - val_accuracy: 0.9847
Epoch 5/10
438/438 [==============================] - 187s 427ms/step - loss: 0.1315 - accuracy: 0.9621 - val_loss: 0.0361 - val_accuracy: 0.9888
Epoch 6/10
438/438 [==============================] - 203s 464ms/step - loss: 0.1145 - accuracy: 0.9679 - val_loss: 0.0367 - val_accuracy: 0.9911
Epoch 7/10
438/438 [==============================] - 203s 463ms/step - loss: 0.0928 - accuracy: 0.9734 - val_loss: 0.032

In [24]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

16
Epoch 1/10
465/465 [==============================] - 219s 462ms/step - loss: 3.3058 - accuracy: 0.1108 - val_loss: 1.2767 - val_accuracy: 0.6504
Epoch 2/10
465/465 [==============================] - 216s 464ms/step - loss: 0.8707 - accuracy: 0.7488 - val_loss: 0.2337 - val_accuracy: 0.9342
Epoch 3/10
465/465 [==============================] - 216s 465ms/step - loss: 0.3398 - accuracy: 0.9001 - val_loss: 0.0722 - val_accuracy: 0.9811
Epoch 4/10
465/465 [==============================] - 216s 464ms/step - loss: 0.2119 - accuracy: 0.9389 - val_loss: 0.0594 - val_accuracy: 0.9852
Epoch 5/10
465/465 [==============================] - 213s 459ms/step - loss: 0.1553 - accuracy: 0.9544 - val_loss: 0.0493 - val_accuracy: 0.9865
Epoch 6/10
465/465 [==============================] - 199s 429ms/step - loss: 0.1332 - accuracy: 0.9633 - val_loss: 0.0495 - val_accuracy: 0.9860
Epoch 7/10
465/465 [==============================] - 198s 426ms/step - loss: 0.1151 - accuracy: 0.9690 - val_loss: 0.036

In [25]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

17
Epoch 1/10
493/493 [==============================] - 214s 426ms/step - loss: 3.2976 - accuracy: 0.1144 - val_loss: 1.2146 - val_accuracy: 0.7316
Epoch 2/10
493/493 [==============================] - 218s 442ms/step - loss: 0.7630 - accuracy: 0.7891 - val_loss: 0.3102 - val_accuracy: 0.9112
Epoch 3/10
493/493 [==============================] - 234s 475ms/step - loss: 0.2484 - accuracy: 0.9263 - val_loss: 0.1292 - val_accuracy: 0.9604
Epoch 4/10
493/493 [==============================] - 237s 481ms/step - loss: 0.1498 - accuracy: 0.9553 - val_loss: 0.0710 - val_accuracy: 0.9804
Epoch 5/10
493/493 [==============================] - 230s 467ms/step - loss: 0.1080 - accuracy: 0.9690 - val_loss: 0.0518 - val_accuracy: 0.9824
Epoch 6/10
493/493 [==============================] - 228s 463ms/step - loss: 0.1007 - accuracy: 0.9715 - val_loss: 0.0248 - val_accuracy: 0.9949
Epoch 7/10
493/493 [==============================] - 232s 471ms/step - loss: 0.0790 - accuracy: 0.9777 - val_loss: 0.029

In [26]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

18
Epoch 1/10
520/520 [==============================] - 227s 429ms/step - loss: 3.3061 - accuracy: 0.1115 - val_loss: 1.1134 - val_accuracy: 0.7157
Epoch 2/10
520/520 [==============================] - 224s 431ms/step - loss: 0.7224 - accuracy: 0.7927 - val_loss: 0.1230 - val_accuracy: 0.9686
Epoch 3/10
520/520 [==============================] - 222s 426ms/step - loss: 0.2112 - accuracy: 0.9379 - val_loss: 0.0413 - val_accuracy: 0.9908
Epoch 4/10
520/520 [==============================] - 220s 423ms/step - loss: 0.1343 - accuracy: 0.9616 - val_loss: 0.0362 - val_accuracy: 0.9923
Epoch 5/10
520/520 [==============================] - 221s 424ms/step - loss: 0.1105 - accuracy: 0.9696 - val_loss: 0.0324 - val_accuracy: 0.9944
Epoch 6/10
520/520 [==============================] - 242s 466ms/step - loss: 0.0927 - accuracy: 0.9739 - val_loss: 0.0307 - val_accuracy: 0.9903
Epoch 7/10
520/520 [==============================] - 242s 465ms/step - loss: 0.0919 - accuracy: 0.9766 - val_loss: 0.046

In [27]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

19
Epoch 1/10
547/547 [==============================] - 258s 463ms/step - loss: 3.3708 - accuracy: 0.1045 - val_loss: 0.7366 - val_accuracy: 0.7686
Epoch 2/10
547/547 [==============================] - 263s 481ms/step - loss: 0.7141 - accuracy: 0.7857 - val_loss: 0.0657 - val_accuracy: 0.9819
Epoch 3/10
547/547 [==============================] - 252s 461ms/step - loss: 0.2057 - accuracy: 0.9390 - val_loss: 0.0416 - val_accuracy: 0.9895
Epoch 4/10
547/547 [==============================] - 253s 462ms/step - loss: 0.1248 - accuracy: 0.9643 - val_loss: 0.0235 - val_accuracy: 0.9939
Epoch 5/10
547/547 [==============================] - 257s 469ms/step - loss: 0.1046 - accuracy: 0.9711 - val_loss: 0.0292 - val_accuracy: 0.9929
Epoch 6/10
547/547 [==============================] - 235s 430ms/step - loss: 0.0906 - accuracy: 0.9747 - val_loss: 0.0348 - val_accuracy: 0.9946
Epoch 7/10
547/547 [==============================] - 226s 413ms/step - loss: 0.0891 - accuracy: 0.9749 - val_loss: 0.021

In [28]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

20
Epoch 1/10
552/552 [==============================] - 237s 424ms/step - loss: 3.2929 - accuracy: 0.1180 - val_loss: 1.1381 - val_accuracy: 0.7211
Epoch 2/10
552/552 [==============================] - 240s 434ms/step - loss: 0.6442 - accuracy: 0.8054 - val_loss: 0.1022 - val_accuracy: 0.9701
Epoch 3/10
552/552 [==============================] - 258s 468ms/step - loss: 0.2035 - accuracy: 0.9397 - val_loss: 0.0338 - val_accuracy: 0.9911
Epoch 4/10
552/552 [==============================] - 256s 465ms/step - loss: 0.1318 - accuracy: 0.9618 - val_loss: 0.0300 - val_accuracy: 0.9941
Epoch 5/10
552/552 [==============================] - 267s 485ms/step - loss: 0.1161 - accuracy: 0.9679 - val_loss: 0.0280 - val_accuracy: 0.9934
Epoch 6/10
552/552 [==============================] - 256s 464ms/step - loss: 0.0879 - accuracy: 0.9758 - val_loss: 0.0299 - val_accuracy: 0.9944
Epoch 7/10
552/552 [==============================] - 261s 472ms/step - loss: 0.0885 - accuracy: 0.9763 - val_loss: 0.025

In [29]:
#rerun 0 model
new_model_dg_dir  = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dg_e1"

i=11

for model in models_dg[11:]:
    model.save(new_model_dg_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dg_e1_11\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dg_e1_12\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dg_e1_13\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dg_e1_14\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dg_e1_15\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dg_e1_16\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dg_e1_17\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dg_e1_18\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gts

In [160]:
#rerun 0 model
new_model_dg_dir  = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dg_e1"

i=0

for model in models_dg[11:12]:
    model.save(new_model_dg_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_dg_e1_0\assets
Model has been saved


In [52]:
loading = False

models_dg = []

if loading:
    for i in range(5):
        model_dg_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_gn_e1_"+str(i)
        print(model_dg_dir)
        model =utils.My_model('gtsrb',True,model_dg_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_dg.append(model)
       
    

In [53]:
try:
    del deep_gini_values
    del top_images_by_dg
    del top_labels_by_dg
    del image_sets_dg
    del label_sets_dg
    del models_dg
except:
    print("Error")

In [55]:
import gc
gc.collect()

100996

### Softmax 

In [125]:
#softmax values
se_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/softmax_values.npy"
se_values = np.load(se_direction)


In [126]:
# Obtaining top n images by LSA values
top_images_by_se  = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),x_train_and_adversary)
top_labels_by_se = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),y_train_and_adversary)

In [127]:
m = n_data_points

n = 0
image_sets_se = []
label_sets_se = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_se)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_se)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_se)%m))
        top_images_by_se_n = np.array(top_images_by_se[:n+m+(len(top_images_by_se)%m)])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m+(len(top_images_by_se)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_se_n = np.array(top_images_by_se[:n+m])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m])
    image_sets_se.append(top_images_by_se_n)
    label_sets_se.append(top_labels_by_se_n)
    print(len(top_images_by_se_n))
    n += m



0 :
0  ->  1750
1750
1 :
0  ->  3500
3500
2 :
0  ->  5250
5250
3 :
0  ->  7000
7000
4 :
0  ->  8750
8750
5 :
0  ->  10500
10500
6 :
0  ->  12250
12250
7 :
0  ->  14000
14000
8 :
0  ->  15750
15750
9 :
0  ->  17500
17500
10 :
0  ->  19250
19250
11 :
0  ->  21000
21000
12 :
0  ->  22750
22750
13 :
0  ->  24500
24500
14 :
0  ->  26250
26250
15 :
0  ->  28000
28000
16 :
0  ->  29750
29750
17 :
0  ->  31500
31500
18 :
0  ->  33250
33250
19 :
0  ->  35000
35000
20 :
Last
0  ->  37037
35287


In [128]:
models_se = []
for i in range(len(label_sets_se)):
    print(i,":")
    model = utils.My_model(dataset,False)
    model.compile_model()
    models_se.append(model)

0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled
20 :
Model compiled


In [61]:
n=0
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

0
Epoch 1/10
28/28 [==============================] - 27s 788ms/step - loss: 3.9488 - accuracy: 0.0781 - val_loss: 3.6190 - val_accuracy: 0.0566
Epoch 2/10
28/28 [==============================] - 22s 794ms/step - loss: 3.4949 - accuracy: 0.1172 - val_loss: 3.5713 - val_accuracy: 0.0566
Epoch 3/10
28/28 [==============================] - 20s 715ms/step - loss: 3.4184 - accuracy: 0.1322 - val_loss: 3.5513 - val_accuracy: 0.0566
Epoch 4/10
28/28 [==============================] - 20s 721ms/step - loss: 3.4157 - accuracy: 0.1183 - val_loss: 3.5434 - val_accuracy: 0.0566
Epoch 5/10
28/28 [==============================] - 20s 734ms/step - loss: 3.4479 - accuracy: 0.1211 - val_loss: 3.5415 - val_accuracy: 0.0566
Epoch 6/10
28/28 [==============================] - 20s 725ms/step - loss: 3.4244 - accuracy: 0.1221 - val_loss: 3.5249 - val_accuracy: 0.0720
Epoch 7/10
28/28 [==============================] - 20s 729ms/step - loss: 3.4101 - accuracy: 0.1194 - val_loss: 3.5946 - val_accuracy: 0.05

In [62]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

1
Epoch 1/10
55/55 [==============================] - 38s 616ms/step - loss: 3.9830 - accuracy: 0.0910 - val_loss: 3.5935 - val_accuracy: 0.0758
Epoch 2/10
55/55 [==============================] - 34s 620ms/step - loss: 3.3757 - accuracy: 0.1282 - val_loss: 3.7574 - val_accuracy: 0.0753
Epoch 3/10
55/55 [==============================] - 34s 611ms/step - loss: 3.3465 - accuracy: 0.1454 - val_loss: 3.6883 - val_accuracy: 0.0783
Epoch 4/10
55/55 [==============================] - 34s 622ms/step - loss: 3.2666 - accuracy: 0.1743 - val_loss: 3.5154 - val_accuracy: 0.0814
Epoch 5/10
55/55 [==============================] - 34s 626ms/step - loss: 3.2057 - accuracy: 0.1629 - val_loss: 3.5806 - val_accuracy: 0.0771
Epoch 6/10
55/55 [==============================] - 33s 598ms/step - loss: 3.0513 - accuracy: 0.1857 - val_loss: 3.2667 - val_accuracy: 0.1212
Epoch 7/10
55/55 [==============================] - 32s 592ms/step - loss: 2.8057 - accuracy: 0.2277 - val_loss: 2.9408 - val_accuracy: 0.20

In [63]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

2
Epoch 1/10
83/83 [==============================] - 51s 563ms/step - loss: 3.5503 - accuracy: 0.0871 - val_loss: 3.9882 - val_accuracy: 0.0541
Epoch 2/10
83/83 [==============================] - 46s 561ms/step - loss: 3.3910 - accuracy: 0.1285 - val_loss: 3.8952 - val_accuracy: 0.0712
Epoch 3/10
83/83 [==============================] - 46s 558ms/step - loss: 3.3330 - accuracy: 0.1497 - val_loss: 3.5178 - val_accuracy: 0.0809
Epoch 4/10
83/83 [==============================] - 46s 559ms/step - loss: 3.1997 - accuracy: 0.1689 - val_loss: 3.4755 - val_accuracy: 0.0855
Epoch 5/10
83/83 [==============================] - 47s 563ms/step - loss: 2.9090 - accuracy: 0.2134 - val_loss: 4.2731 - val_accuracy: 0.1013
Epoch 6/10
83/83 [==============================] - 47s 561ms/step - loss: 2.5470 - accuracy: 0.2978 - val_loss: 2.5898 - val_accuracy: 0.3121
Epoch 7/10
83/83 [==============================] - 47s 565ms/step - loss: 2.1331 - accuracy: 0.3926 - val_loss: 1.9077 - val_accuracy: 0.47

In [64]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

3
Epoch 1/10
110/110 [==============================] - 68s 577ms/step - loss: 3.5633 - accuracy: 0.0895 - val_loss: 3.7173 - val_accuracy: 0.0727
Epoch 2/10
110/110 [==============================] - 64s 581ms/step - loss: 3.4040 - accuracy: 0.1227 - val_loss: 3.6899 - val_accuracy: 0.0755
Epoch 3/10
110/110 [==============================] - 60s 544ms/step - loss: 3.3150 - accuracy: 0.1507 - val_loss: 3.4888 - val_accuracy: 0.0873
Epoch 4/10
110/110 [==============================] - 60s 543ms/step - loss: 3.1198 - accuracy: 0.1684 - val_loss: 3.1550 - val_accuracy: 0.1460
Epoch 5/10
110/110 [==============================] - 60s 549ms/step - loss: 2.6383 - accuracy: 0.2607 - val_loss: 2.7204 - val_accuracy: 0.2840
Epoch 6/10
110/110 [==============================] - 60s 549ms/step - loss: 2.1014 - accuracy: 0.3902 - val_loss: 1.5654 - val_accuracy: 0.5588
Epoch 7/10
110/110 [==============================] - 60s 550ms/step - loss: 1.5444 - accuracy: 0.5422 - val_loss: 1.2454 - val_

In [65]:
print(n)#4
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

4
Epoch 1/10
137/137 [==============================] - 78s 537ms/step - loss: 3.9204 - accuracy: 0.0903 - val_loss: 3.6328 - val_accuracy: 0.0809
Epoch 2/10
137/137 [==============================] - 73s 531ms/step - loss: 3.3482 - accuracy: 0.1452 - val_loss: 3.3668 - val_accuracy: 0.1153
Epoch 3/10
137/137 [==============================] - 73s 533ms/step - loss: 2.9644 - accuracy: 0.2005 - val_loss: 2.6968 - val_accuracy: 0.2220
Epoch 4/10
137/137 [==============================] - 73s 533ms/step - loss: 2.2180 - accuracy: 0.3313 - val_loss: 1.5039 - val_accuracy: 0.5708
Epoch 5/10
137/137 [==============================] - 74s 538ms/step - loss: 1.5294 - accuracy: 0.5444 - val_loss: 0.8435 - val_accuracy: 0.7400
Epoch 6/10
137/137 [==============================] - 74s 538ms/step - loss: 0.9878 - accuracy: 0.6969 - val_loss: 0.5213 - val_accuracy: 0.8408
Epoch 7/10
137/137 [==============================] - 73s 534ms/step - loss: 0.7161 - accuracy: 0.7790 - val_loss: 0.2751 - val_

In [66]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

5
Epoch 1/10
165/165 [==============================] - 92s 526ms/step - loss: 3.5908 - accuracy: 0.0952 - val_loss: 3.8952 - val_accuracy: 0.0722
Epoch 2/10
165/165 [==============================] - 86s 522ms/step - loss: 3.3314 - accuracy: 0.1542 - val_loss: 3.3574 - val_accuracy: 0.1146
Epoch 3/10
165/165 [==============================] - 87s 525ms/step - loss: 2.7892 - accuracy: 0.2355 - val_loss: 2.2240 - val_accuracy: 0.4164
Epoch 4/10
165/165 [==============================] - 87s 526ms/step - loss: 1.7998 - accuracy: 0.4694 - val_loss: 1.0757 - val_accuracy: 0.6798
Epoch 5/10
165/165 [==============================] - 87s 527ms/step - loss: 1.0523 - accuracy: 0.6817 - val_loss: 0.7294 - val_accuracy: 0.7864
Epoch 6/10
165/165 [==============================] - 87s 525ms/step - loss: 0.6718 - accuracy: 0.7966 - val_loss: 0.2547 - val_accuracy: 0.9183
Epoch 7/10
165/165 [==============================] - 90s 543ms/step - loss: 0.4673 - accuracy: 0.8590 - val_loss: 0.1447 - val_

In [67]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

6
Epoch 1/10
192/192 [==============================] - 105s 522ms/step - loss: 3.5454 - accuracy: 0.0948 - val_loss: 3.5734 - val_accuracy: 0.0855
Epoch 2/10
192/192 [==============================] - 99s 518ms/step - loss: 3.2066 - accuracy: 0.1661 - val_loss: 2.5587 - val_accuracy: 0.2919
Epoch 3/10
192/192 [==============================] - 100s 521ms/step - loss: 2.1454 - accuracy: 0.3743 - val_loss: 1.3036 - val_accuracy: 0.6351
Epoch 4/10
192/192 [==============================] - 100s 521ms/step - loss: 1.1686 - accuracy: 0.6575 - val_loss: 0.5501 - val_accuracy: 0.8232
Epoch 5/10
192/192 [==============================] - 100s 520ms/step - loss: 0.6621 - accuracy: 0.7958 - val_loss: 0.3803 - val_accuracy: 0.9074
Epoch 6/10
192/192 [==============================] - 100s 523ms/step - loss: 0.4765 - accuracy: 0.8585 - val_loss: 0.1376 - val_accuracy: 0.9582
Epoch 7/10
192/192 [==============================] - 100s 521ms/step - loss: 0.3355 - accuracy: 0.8973 - val_loss: 0.1182 

In [68]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

7
Epoch 1/10
219/219 [==============================] - 118s 518ms/step - loss: 3.5765 - accuracy: 0.0904 - val_loss: 3.5624 - val_accuracy: 0.0845
Epoch 2/10
219/219 [==============================] - 113s 516ms/step - loss: 3.2653 - accuracy: 0.1484 - val_loss: 2.5220 - val_accuracy: 0.2860
Epoch 3/10
219/219 [==============================] - 114s 520ms/step - loss: 2.1729 - accuracy: 0.3608 - val_loss: 0.9658 - val_accuracy: 0.7022
Epoch 4/10
219/219 [==============================] - 114s 521ms/step - loss: 1.0744 - accuracy: 0.6745 - val_loss: 0.3815 - val_accuracy: 0.8857
Epoch 5/10
219/219 [==============================] - 115s 523ms/step - loss: 0.6343 - accuracy: 0.8080 - val_loss: 0.1827 - val_accuracy: 0.9485
Epoch 6/10
219/219 [==============================] - 114s 521ms/step - loss: 0.3894 - accuracy: 0.8848 - val_loss: 0.1047 - val_accuracy: 0.9681
Epoch 7/10
219/219 [==============================] - 115s 524ms/step - loss: 0.2631 - accuracy: 0.9177 - val_loss: 0.0903

In [69]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

8
Epoch 1/10
247/247 [==============================] - 131s 513ms/step - loss: 3.6197 - accuracy: 0.0945 - val_loss: 3.5166 - val_accuracy: 0.1028
Epoch 2/10
247/247 [==============================] - 127s 515ms/step - loss: 2.7837 - accuracy: 0.2306 - val_loss: 1.7728 - val_accuracy: 0.5461
Epoch 3/10
247/247 [==============================] - 127s 516ms/step - loss: 1.2483 - accuracy: 0.6250 - val_loss: 0.2766 - val_accuracy: 0.9181
Epoch 4/10
247/247 [==============================] - 127s 515ms/step - loss: 0.5705 - accuracy: 0.8222 - val_loss: 0.1974 - val_accuracy: 0.9479
Epoch 5/10
247/247 [==============================] - 130s 528ms/step - loss: 0.3639 - accuracy: 0.8902 - val_loss: 0.0947 - val_accuracy: 0.9796
Epoch 6/10
247/247 [==============================] - 128s 517ms/step - loss: 0.2413 - accuracy: 0.9283 - val_loss: 0.0497 - val_accuracy: 0.9844
Epoch 7/10
247/247 [==============================] - 127s 515ms/step - loss: 0.2137 - accuracy: 0.9401 - val_loss: 0.0285

In [70]:
print(n)#9
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

9
Epoch 1/10
274/274 [==============================] - 143s 508ms/step - loss: 3.6063 - accuracy: 0.0969 - val_loss: 3.1809 - val_accuracy: 0.1324
Epoch 2/10
274/274 [==============================] - 140s 512ms/step - loss: 2.6718 - accuracy: 0.2525 - val_loss: 1.1300 - val_accuracy: 0.7553
Epoch 3/10
274/274 [==============================] - 140s 511ms/step - loss: 1.0731 - accuracy: 0.6707 - val_loss: 0.2214 - val_accuracy: 0.9416
Epoch 4/10
274/274 [==============================] - 141s 516ms/step - loss: 0.5004 - accuracy: 0.8431 - val_loss: 0.1799 - val_accuracy: 0.9566
Epoch 5/10
274/274 [==============================] - 140s 511ms/step - loss: 0.3004 - accuracy: 0.9119 - val_loss: 0.0495 - val_accuracy: 0.9880
Epoch 6/10
274/274 [==============================] - 140s 511ms/step - loss: 0.2004 - accuracy: 0.9391 - val_loss: 0.0442 - val_accuracy: 0.9885
Epoch 7/10
274/274 [==============================] - 140s 509ms/step - loss: 0.1654 - accuracy: 0.9525 - val_loss: 0.0282

In [73]:
n=10
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

10
Epoch 1/10
301/301 [==============================] - 138s 443ms/step - loss: 3.5451 - accuracy: 0.0925 - val_loss: 2.9733 - val_accuracy: 0.1891
Epoch 2/10
301/301 [==============================] - 135s 449ms/step - loss: 2.3918 - accuracy: 0.3197 - val_loss: 0.4968 - val_accuracy: 0.8446
Epoch 3/10
301/301 [==============================] - 137s 454ms/step - loss: 0.8427 - accuracy: 0.7459 - val_loss: 0.1234 - val_accuracy: 0.9658
Epoch 4/10
301/301 [==============================] - 140s 465ms/step - loss: 0.4350 - accuracy: 0.8676 - val_loss: 0.0758 - val_accuracy: 0.9798
Epoch 5/10
301/301 [==============================] - 145s 482ms/step - loss: 0.2731 - accuracy: 0.9156 - val_loss: 0.0502 - val_accuracy: 0.9875
Epoch 6/10
301/301 [==============================] - 152s 505ms/step - loss: 0.1956 - accuracy: 0.9404 - val_loss: 0.0383 - val_accuracy: 0.9888
Epoch 7/10
301/301 [==============================] - 158s 524ms/step - loss: 0.1652 - accuracy: 0.9501 - val_loss: 0.042

In [74]:
n=11
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

11
Epoch 1/10
329/329 [==============================] - 175s 519ms/step - loss: 3.5451 - accuracy: 0.0877 - val_loss: 3.0954 - val_accuracy: 0.1615
Epoch 2/10
329/329 [==============================] - 168s 511ms/step - loss: 2.2026 - accuracy: 0.3647 - val_loss: 0.4381 - val_accuracy: 0.8398
Epoch 3/10
329/329 [==============================] - 170s 517ms/step - loss: 0.6953 - accuracy: 0.7835 - val_loss: 0.1525 - val_accuracy: 0.9569
Epoch 4/10
329/329 [==============================] - 170s 517ms/step - loss: 0.3372 - accuracy: 0.8990 - val_loss: 0.0737 - val_accuracy: 0.9796
Epoch 5/10
329/329 [==============================] - 170s 517ms/step - loss: 0.2189 - accuracy: 0.9322 - val_loss: 0.0371 - val_accuracy: 0.9895
Epoch 6/10
329/329 [==============================] - 170s 517ms/step - loss: 0.1603 - accuracy: 0.9517 - val_loss: 0.0353 - val_accuracy: 0.9898
Epoch 7/10
329/329 [==============================] - 168s 512ms/step - loss: 0.1361 - accuracy: 0.9579 - val_loss: 0.022

In [75]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

12
Epoch 1/10
356/356 [==============================] - 189s 518ms/step - loss: 3.5494 - accuracy: 0.0844 - val_loss: 2.3368 - val_accuracy: 0.3516
Epoch 2/10
356/356 [==============================] - 223s 626ms/step - loss: 1.8974 - accuracy: 0.4381 - val_loss: 0.2946 - val_accuracy: 0.9020
Epoch 3/10
356/356 [==============================] - 246s 690ms/step - loss: 0.5509 - accuracy: 0.8294 - val_loss: 0.1272 - val_accuracy: 0.9630
Epoch 4/10
356/356 [==============================] - 245s 689ms/step - loss: 0.2770 - accuracy: 0.9161 - val_loss: 0.0371 - val_accuracy: 0.9890
Epoch 5/10
356/356 [==============================] - 245s 688ms/step - loss: 0.1814 - accuracy: 0.9473 - val_loss: 0.0455 - val_accuracy: 0.9883
Epoch 6/10
356/356 [==============================] - 245s 690ms/step - loss: 0.1424 - accuracy: 0.9566 - val_loss: 0.0204 - val_accuracy: 0.9964
Epoch 7/10
356/356 [==============================] - 245s 688ms/step - loss: 0.1203 - accuracy: 0.9661 - val_loss: 0.069

In [129]:
n=13
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

13
Epoch 1/10
383/383 [==============================] - 279s 705ms/step - loss: 3.4529 - accuracy: 0.0979 - val_loss: 1.1713 - val_accuracy: 0.6685
Epoch 2/10
383/383 [==============================] - 270s 705ms/step - loss: 1.2065 - accuracy: 0.6368 - val_loss: 0.2417 - val_accuracy: 0.9293
Epoch 3/10
383/383 [==============================] - 268s 700ms/step - loss: 0.4042 - accuracy: 0.8743 - val_loss: 0.0698 - val_accuracy: 0.9793
Epoch 4/10
383/383 [==============================] - 268s 700ms/step - loss: 0.2299 - accuracy: 0.9316 - val_loss: 0.0309 - val_accuracy: 0.9921
Epoch 5/10
383/383 [==============================] - 268s 699ms/step - loss: 0.1653 - accuracy: 0.9523 - val_loss: 0.0302 - val_accuracy: 0.9906
Epoch 6/10
383/383 [==============================] - 268s 701ms/step - loss: 0.1340 - accuracy: 0.9611 - val_loss: 0.0248 - val_accuracy: 0.9934
Epoch 7/10
383/383 [==============================] - 268s 701ms/step - loss: 0.1117 - accuracy: 0.9677 - val_loss: 0.022

In [130]:
print(n)#14
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

14
Epoch 1/10
411/411 [==============================] - 296s 697ms/step - loss: 3.5548 - accuracy: 0.0829 - val_loss: 1.9908 - val_accuracy: 0.4231
Epoch 2/10
411/411 [==============================] - 284s 691ms/step - loss: 1.6197 - accuracy: 0.5107 - val_loss: 0.2076 - val_accuracy: 0.9367
Epoch 3/10
411/411 [==============================] - 284s 692ms/step - loss: 0.4971 - accuracy: 0.8458 - val_loss: 0.0717 - val_accuracy: 0.9816
Epoch 4/10
411/411 [==============================] - 283s 689ms/step - loss: 0.2524 - accuracy: 0.9245 - val_loss: 0.0832 - val_accuracy: 0.9750
Epoch 5/10
411/411 [==============================] - 284s 691ms/step - loss: 0.1852 - accuracy: 0.9461 - val_loss: 0.0326 - val_accuracy: 0.9908
Epoch 6/10
411/411 [==============================] - 285s 694ms/step - loss: 0.1359 - accuracy: 0.9593 - val_loss: 0.1931 - val_accuracy: 0.9579
Epoch 7/10
411/411 [==============================] - 284s 690ms/step - loss: 0.1335 - accuracy: 0.9617 - val_loss: 0.050

In [131]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

15
Epoch 1/10
438/438 [==============================] - 316s 700ms/step - loss: 3.4783 - accuracy: 0.0958 - val_loss: 1.6685 - val_accuracy: 0.6004
Epoch 2/10
438/438 [==============================] - 307s 701ms/step - loss: 1.1040 - accuracy: 0.6647 - val_loss: 0.1494 - val_accuracy: 0.9584
Epoch 3/10
438/438 [==============================] - 306s 699ms/step - loss: 0.3345 - accuracy: 0.9015 - val_loss: 0.2035 - val_accuracy: 0.9385
Epoch 4/10
438/438 [==============================] - 306s 699ms/step - loss: 0.2103 - accuracy: 0.9386 - val_loss: 0.0394 - val_accuracy: 0.9893
Epoch 5/10
438/438 [==============================] - 307s 701ms/step - loss: 0.1412 - accuracy: 0.9595 - val_loss: 0.0370 - val_accuracy: 0.9923
Epoch 6/10
438/438 [==============================] - 308s 702ms/step - loss: 0.1108 - accuracy: 0.9676 - val_loss: 0.0377 - val_accuracy: 0.9934
Epoch 7/10
438/438 [==============================] - 265s 606ms/step - loss: 0.0961 - accuracy: 0.9718 - val_loss: 0.043

In [132]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

16
Epoch 1/10
465/465 [==============================] - 245s 519ms/step - loss: 3.4504 - accuracy: 0.0970 - val_loss: 0.8577 - val_accuracy: 0.7441
Epoch 2/10
465/465 [==============================] - 298s 641ms/step - loss: 0.9815 - accuracy: 0.6979 - val_loss: 0.1017 - val_accuracy: 0.9709
Epoch 3/10
465/465 [==============================] - 331s 712ms/step - loss: 0.2984 - accuracy: 0.9137 - val_loss: 0.0796 - val_accuracy: 0.9791
Epoch 4/10
465/465 [==============================] - 314s 675ms/step - loss: 0.1761 - accuracy: 0.9491 - val_loss: 0.0272 - val_accuracy: 0.9923
Epoch 5/10
465/465 [==============================] - 313s 673ms/step - loss: 0.1218 - accuracy: 0.9646 - val_loss: 0.0218 - val_accuracy: 0.9949
Epoch 6/10
465/465 [==============================] - 314s 675ms/step - loss: 0.1171 - accuracy: 0.9667 - val_loss: 0.0171 - val_accuracy: 0.9959
Epoch 7/10
465/465 [==============================] - 312s 672ms/step - loss: 0.1034 - accuracy: 0.9720 - val_loss: 0.032

In [133]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

17
Epoch 1/10
493/493 [==============================] - 338s 676ms/step - loss: 3.4221 - accuracy: 0.0995 - val_loss: 1.3497 - val_accuracy: 0.6890
Epoch 2/10
493/493 [==============================] - 224s 455ms/step - loss: 0.9357 - accuracy: 0.7173 - val_loss: 0.2527 - val_accuracy: 0.9342
Epoch 3/10
493/493 [==============================] - 231s 468ms/step - loss: 0.3068 - accuracy: 0.9127 - val_loss: 0.0360 - val_accuracy: 0.9916
Epoch 4/10
493/493 [==============================] - 244s 495ms/step - loss: 0.1797 - accuracy: 0.9476 - val_loss: 0.0427 - val_accuracy: 0.9903
Epoch 5/10
493/493 [==============================] - 260s 528ms/step - loss: 0.1395 - accuracy: 0.9611 - val_loss: 0.0317 - val_accuracy: 0.9926
Epoch 6/10
493/493 [==============================] - 258s 524ms/step - loss: 0.1249 - accuracy: 0.9656 - val_loss: 0.0227 - val_accuracy: 0.9936
Epoch 7/10
493/493 [==============================] - 257s 522ms/step - loss: 0.0997 - accuracy: 0.9729 - val_loss: 0.034

In [134]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

18
Epoch 1/10
520/520 [==============================] - 274s 519ms/step - loss: 3.4378 - accuracy: 0.0868 - val_loss: 1.1251 - val_accuracy: 0.7344
Epoch 2/10
520/520 [==============================] - 273s 525ms/step - loss: 0.9836 - accuracy: 0.6984 - val_loss: 0.1824 - val_accuracy: 0.9497
Epoch 3/10
520/520 [==============================] - 277s 533ms/step - loss: 0.2867 - accuracy: 0.9143 - val_loss: 0.0805 - val_accuracy: 0.9781
Epoch 4/10
520/520 [==============================] - 270s 519ms/step - loss: 0.1753 - accuracy: 0.9507 - val_loss: 0.0377 - val_accuracy: 0.9934
Epoch 5/10
520/520 [==============================] - 269s 518ms/step - loss: 0.1217 - accuracy: 0.9656 - val_loss: 0.0310 - val_accuracy: 0.9921
Epoch 6/10
520/520 [==============================] - 274s 527ms/step - loss: 0.1118 - accuracy: 0.9690 - val_loss: 0.0259 - val_accuracy: 0.9939
Epoch 7/10
520/520 [==============================] - 260s 500ms/step - loss: 0.0927 - accuracy: 0.9748 - val_loss: 0.025

In [135]:
print(n)#19
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

19
Epoch 1/10
547/547 [==============================] - 288s 519ms/step - loss: 3.2261 - accuracy: 0.1425 - val_loss: 0.4447 - val_accuracy: 0.8655
Epoch 2/10
547/547 [==============================] - 285s 520ms/step - loss: 0.6118 - accuracy: 0.8186 - val_loss: 0.0685 - val_accuracy: 0.9819
Epoch 3/10
547/547 [==============================] - 283s 517ms/step - loss: 0.2202 - accuracy: 0.9362 - val_loss: 0.0345 - val_accuracy: 0.9913
Epoch 4/10
547/547 [==============================] - 284s 520ms/step - loss: 0.1390 - accuracy: 0.9598 - val_loss: 0.0370 - val_accuracy: 0.9923
Epoch 5/10
547/547 [==============================] - 283s 517ms/step - loss: 0.1089 - accuracy: 0.9683 - val_loss: 0.0348 - val_accuracy: 0.9921
Epoch 6/10
547/547 [==============================] - 283s 517ms/step - loss: 0.0946 - accuracy: 0.9747 - val_loss: 0.0309 - val_accuracy: 0.9926
Epoch 7/10
547/547 [==============================] - 286s 523ms/step - loss: 0.0890 - accuracy: 0.9751 - val_loss: 0.028

In [136]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

20
Epoch 1/10
552/552 [==============================] - 307s 547ms/step - loss: 3.3352 - accuracy: 0.1139 - val_loss: 0.5678 - val_accuracy: 0.8285
Epoch 2/10
552/552 [==============================] - 296s 536ms/step - loss: 0.6639 - accuracy: 0.8046 - val_loss: 0.1038 - val_accuracy: 0.9750
Epoch 3/10
552/552 [==============================] - 288s 522ms/step - loss: 0.2211 - accuracy: 0.9359 - val_loss: 0.0361 - val_accuracy: 0.9906
Epoch 4/10
552/552 [==============================] - 286s 519ms/step - loss: 0.1519 - accuracy: 0.9560 - val_loss: 0.0377 - val_accuracy: 0.9893
Epoch 5/10
552/552 [==============================] - 289s 523ms/step - loss: 0.1114 - accuracy: 0.9689 - val_loss: 0.0244 - val_accuracy: 0.9944
Epoch 6/10
552/552 [==============================] - 286s 519ms/step - loss: 0.1019 - accuracy: 0.9731 - val_loss: 0.0254 - val_accuracy: 0.9952
Epoch 7/10
552/552 [==============================] - 286s 518ms/step - loss: 0.0940 - accuracy: 0.9747 - val_loss: 0.034

In [72]:
len(models_se)

21

In [137]:
new_model_se_dir  = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_se_e1"

i=13

for model in models_se[13:]:
    model.save(new_model_se_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_se_e1_13\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_se_e1_14\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_se_e1_15\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_se_e1_16\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_se_e1_17\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_se_e1_18\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_se_e1_19\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_se_e1_20\assets
Model has been saved


In [138]:
try:
    del se_values
    del top_images_by_se
    del top_labels_by_se
    del image_sets_se
    del label_sets_se
    del models_se
except:
    print("Error")

In [139]:
gc.collect()

73414

## Training guided by Random values

In [ ]:
import random
random_indexes =list(range(len(x_train_and_adversary)))
random.shuffle(random_indexes)
print(random_indexes[:10])
print(len(random_indexes))

In [15]:
save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/random_values.npy"


random_indexes = np.load(save_dir)

In [16]:
len(random_indexes)

35287

In [17]:
# Obtaining top n images by random values
top_images_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),x_train_and_adversary)
top_labels_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),y_train_and_adversary)

In [18]:
m = n_data_points
n = 0
image_sets_random = []
label_sets_random = []



for i in range((len(top_images_by_random)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_random)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_random)%m))
        top_images_by_random_n = np.array(top_images_by_random[:n+m+(len(top_images_by_random)%m)])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m+(len(top_images_by_random)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_random_n = np.array(top_images_by_random[:n+m])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m])
    image_sets_random.append(top_images_by_random_n)
    label_sets_random.append(top_labels_by_random_n)
    print(len(top_images_by_random_n))
    n += m




0 :
0  ->  1750
1750
1 :
0  ->  3500
3500
2 :
0  ->  5250
5250
3 :
0  ->  7000
7000
4 :
0  ->  8750
8750
5 :
0  ->  10500
10500
6 :
0  ->  12250
12250
7 :
0  ->  14000
14000
8 :
0  ->  15750
15750
9 :
0  ->  17500
17500
10 :
0  ->  19250
19250
11 :
0  ->  21000
21000
12 :
0  ->  22750
22750
13 :
0  ->  24500
24500
14 :
0  ->  26250
26250
15 :
0  ->  28000
28000
16 :
0  ->  29750
29750
17 :
0  ->  31500
31500
18 :
0  ->  33250
33250
19 :
0  ->  35000
35000
20 :
Last
0  ->  37037
35287


In [21]:
len(image_sets_random)

21

In [20]:
print(model_dir)

models_random = []
for i in range(len(label_sets_random)):
    print(i,":")
    model = utils.My_model(dataset,False,None)
    model.compile_model()
    models_random.append(model)


C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/
0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled
20 :
Model compiled


In [145]:
n=0
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

0
Epoch 1/10
28/28 [==============================] - 41s 1s/step - loss: 3.7066 - accuracy: 0.0474 - val_loss: 3.5053 - val_accuracy: 0.0646
Epoch 2/10
28/28 [==============================] - 29s 1s/step - loss: 3.5529 - accuracy: 0.0665 - val_loss: 3.5015 - val_accuracy: 0.0551
Epoch 3/10
28/28 [==============================] - 29s 1s/step - loss: 3.5464 - accuracy: 0.0594 - val_loss: 3.4888 - val_accuracy: 0.0635
Epoch 4/10
28/28 [==============================] - 29s 1s/step - loss: 3.5047 - accuracy: 0.0808 - val_loss: 3.4819 - val_accuracy: 0.0799
Epoch 5/10
28/28 [==============================] - 28s 1s/step - loss: 3.4605 - accuracy: 0.0888 - val_loss: 3.4365 - val_accuracy: 0.0926
Epoch 6/10
28/28 [==============================] - 29s 1s/step - loss: 3.4413 - accuracy: 0.0771 - val_loss: 3.3993 - val_accuracy: 0.0965
Epoch 7/10
28/28 [==============================] - 29s 1s/step - loss: 3.4420 - accuracy: 0.0761 - val_loss: 3.4080 - val_accuracy: 0.0901
Epoch 8/10
28/28 [

In [146]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

1
Epoch 1/10
55/55 [==============================] - 58s 886ms/step - loss: 3.6938 - accuracy: 0.0489 - val_loss: 3.5070 - val_accuracy: 0.0551
Epoch 2/10
55/55 [==============================] - 47s 857ms/step - loss: 3.5424 - accuracy: 0.0511 - val_loss: 3.5100 - val_accuracy: 0.0845
Epoch 3/10
55/55 [==============================] - 48s 868ms/step - loss: 3.4920 - accuracy: 0.0796 - val_loss: 3.4178 - val_accuracy: 0.0947
Epoch 4/10
55/55 [==============================] - 48s 867ms/step - loss: 3.4380 - accuracy: 0.0740 - val_loss: 3.3786 - val_accuracy: 0.0993
Epoch 5/10
55/55 [==============================] - 47s 858ms/step - loss: 3.3231 - accuracy: 0.1131 - val_loss: 3.1201 - val_accuracy: 0.1536
Epoch 6/10
55/55 [==============================] - 47s 852ms/step - loss: 2.9579 - accuracy: 0.2161 - val_loss: 2.2699 - val_accuracy: 0.3514
Epoch 7/10
55/55 [==============================] - 47s 857ms/step - loss: 2.3440 - accuracy: 0.3553 - val_loss: 1.7441 - val_accuracy: 0.46

In [147]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

2
Epoch 1/10
83/83 [==============================] - 65s 664ms/step - loss: 3.6517 - accuracy: 0.0483 - val_loss: 3.6066 - val_accuracy: 0.0498
Epoch 2/10
83/83 [==============================] - 41s 498ms/step - loss: 3.5164 - accuracy: 0.0789 - val_loss: 3.3947 - val_accuracy: 0.0817
Epoch 3/10
83/83 [==============================] - 43s 513ms/step - loss: 3.4279 - accuracy: 0.0926 - val_loss: 3.2195 - val_accuracy: 0.1227
Epoch 4/10
83/83 [==============================] - 45s 546ms/step - loss: 3.1278 - accuracy: 0.1580 - val_loss: 2.6620 - val_accuracy: 0.2534
Epoch 5/10
83/83 [==============================] - 52s 631ms/step - loss: 2.3981 - accuracy: 0.3391 - val_loss: 1.7847 - val_accuracy: 0.4649
Epoch 6/10
83/83 [==============================] - 43s 513ms/step - loss: 1.7826 - accuracy: 0.4747 - val_loss: 1.3127 - val_accuracy: 0.5690
Epoch 7/10
83/83 [==============================] - 42s 511ms/step - loss: 1.2849 - accuracy: 0.6021 - val_loss: 0.8516 - val_accuracy: 0.72

In [22]:
n=3
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

3
Epoch 1/10
110/110 [==============================] - 95s 568ms/step - loss: 3.6506 - accuracy: 0.0481 - val_loss: 3.4113 - val_accuracy: 0.0944
Epoch 2/10
110/110 [==============================] - 58s 532ms/step - loss: 3.4575 - accuracy: 0.0777 - val_loss: 3.2246 - val_accuracy: 0.1312
Epoch 3/10
110/110 [==============================] - 59s 539ms/step - loss: 3.1040 - accuracy: 0.1733 - val_loss: 2.3455 - val_accuracy: 0.3465
Epoch 4/10
110/110 [==============================] - 61s 557ms/step - loss: 2.0770 - accuracy: 0.4267 - val_loss: 0.9986 - val_accuracy: 0.6920
Epoch 5/10
110/110 [==============================] - 59s 540ms/step - loss: 1.2563 - accuracy: 0.6222 - val_loss: 0.5617 - val_accuracy: 0.8102
Epoch 6/10
110/110 [==============================] - 59s 540ms/step - loss: 0.7882 - accuracy: 0.7741 - val_loss: 0.3505 - val_accuracy: 0.8941
Epoch 7/10
110/110 [==============================] - 64s 582ms/step - loss: 0.5657 - accuracy: 0.8347 - val_loss: 0.1891 - val_

In [23]:
print(n)#4

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

4
Epoch 1/10
137/137 [==============================] - 77s 529ms/step - loss: 3.6223 - accuracy: 0.0449 - val_loss: 3.5088 - val_accuracy: 0.0817
Epoch 2/10
137/137 [==============================] - 72s 523ms/step - loss: 3.4874 - accuracy: 0.0703 - val_loss: 3.3476 - val_accuracy: 0.1041
Epoch 3/10
137/137 [==============================] - 75s 546ms/step - loss: 3.2072 - accuracy: 0.1420 - val_loss: 1.9064 - val_accuracy: 0.4565
Epoch 4/10
137/137 [==============================] - 74s 543ms/step - loss: 1.9896 - accuracy: 0.4458 - val_loss: 0.8516 - val_accuracy: 0.7535
Epoch 5/10
137/137 [==============================] - 72s 530ms/step - loss: 1.0896 - accuracy: 0.6681 - val_loss: 0.3790 - val_accuracy: 0.9069
Epoch 6/10
137/137 [==============================] - 72s 529ms/step - loss: 0.6543 - accuracy: 0.8035 - val_loss: 0.2691 - val_accuracy: 0.9252
Epoch 7/10
137/137 [==============================] - 73s 531ms/step - loss: 0.4233 - accuracy: 0.8665 - val_loss: 0.1640 - val_

In [24]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

5
Epoch 1/10
165/165 [==============================] - 90s 517ms/step - loss: 3.6327 - accuracy: 0.0569 - val_loss: 3.4122 - val_accuracy: 0.0857
Epoch 2/10
165/165 [==============================] - 85s 517ms/step - loss: 3.3485 - accuracy: 0.1050 - val_loss: 2.1377 - val_accuracy: 0.3960
Epoch 3/10
165/165 [==============================] - 86s 519ms/step - loss: 2.1088 - accuracy: 0.4075 - val_loss: 1.0731 - val_accuracy: 0.7071
Epoch 4/10
165/165 [==============================] - 87s 528ms/step - loss: 1.0572 - accuracy: 0.6858 - val_loss: 0.4076 - val_accuracy: 0.8711
Epoch 5/10
165/165 [==============================] - 86s 519ms/step - loss: 0.5672 - accuracy: 0.8296 - val_loss: 0.5670 - val_accuracy: 0.9168
Epoch 6/10
165/165 [==============================] - 85s 518ms/step - loss: 0.3739 - accuracy: 0.8842 - val_loss: 0.1207 - val_accuracy: 0.9678
Epoch 7/10
165/165 [==============================] - 86s 520ms/step - loss: 0.2532 - accuracy: 0.9219 - val_loss: 0.0809 - val_

In [25]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

6
Epoch 1/10
192/192 [==============================] - 103s 513ms/step - loss: 3.5964 - accuracy: 0.0515 - val_loss: 3.4921 - val_accuracy: 0.0704
Epoch 2/10
192/192 [==============================] - 98s 512ms/step - loss: 3.3390 - accuracy: 0.1099 - val_loss: 2.0027 - val_accuracy: 0.4330
Epoch 3/10
192/192 [==============================] - 99s 514ms/step - loss: 1.9914 - accuracy: 0.4391 - val_loss: 0.7448 - val_accuracy: 0.7887
Epoch 4/10
192/192 [==============================] - 100s 519ms/step - loss: 0.9494 - accuracy: 0.7148 - val_loss: 0.2985 - val_accuracy: 0.9196
Epoch 5/10
192/192 [==============================] - 101s 524ms/step - loss: 0.5122 - accuracy: 0.8504 - val_loss: 0.1994 - val_accuracy: 0.9441
Epoch 6/10
192/192 [==============================] - 99s 516ms/step - loss: 0.3044 - accuracy: 0.9075 - val_loss: 0.1149 - val_accuracy: 0.9717
Epoch 7/10
192/192 [==============================] - 99s 514ms/step - loss: 0.2212 - accuracy: 0.9354 - val_loss: 0.0857 - v

In [26]:
n=7
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

7
Epoch 1/10
219/219 [==============================] - 116s 512ms/step - loss: 3.5690 - accuracy: 0.0544 - val_loss: 3.2607 - val_accuracy: 0.1477
Epoch 2/10
219/219 [==============================] - 112s 511ms/step - loss: 2.8455 - accuracy: 0.2233 - val_loss: 0.9605 - val_accuracy: 0.7058
Epoch 3/10
219/219 [==============================] - 113s 515ms/step - loss: 1.0922 - accuracy: 0.6644 - val_loss: 0.2130 - val_accuracy: 0.9385
Epoch 4/10
219/219 [==============================] - 112s 511ms/step - loss: 0.4722 - accuracy: 0.8546 - val_loss: 0.1227 - val_accuracy: 0.9635
Epoch 5/10
219/219 [==============================] - 112s 511ms/step - loss: 0.2839 - accuracy: 0.9151 - val_loss: 0.0697 - val_accuracy: 0.9814
Epoch 6/10
219/219 [==============================] - 112s 511ms/step - loss: 0.1844 - accuracy: 0.9429 - val_loss: 0.0532 - val_accuracy: 0.9860
Epoch 7/10
219/219 [==============================] - 112s 511ms/step - loss: 0.1435 - accuracy: 0.9560 - val_loss: 0.0545

In [27]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

8
Epoch 1/10
247/247 [==============================] - 131s 512ms/step - loss: 3.5714 - accuracy: 0.0531 - val_loss: 3.4035 - val_accuracy: 0.0860
Epoch 2/10
247/247 [==============================] - 124s 504ms/step - loss: 3.3084 - accuracy: 0.1105 - val_loss: 1.9097 - val_accuracy: 0.4486
Epoch 3/10
247/247 [==============================] - 126s 509ms/step - loss: 1.6653 - accuracy: 0.5078 - val_loss: 0.5053 - val_accuracy: 0.8449
Epoch 4/10
247/247 [==============================] - 125s 508ms/step - loss: 0.6614 - accuracy: 0.7911 - val_loss: 0.2465 - val_accuracy: 0.9303
Epoch 5/10
247/247 [==============================] - 125s 507ms/step - loss: 0.3355 - accuracy: 0.8997 - val_loss: 0.0667 - val_accuracy: 0.9801
Epoch 6/10
247/247 [==============================] - 125s 506ms/step - loss: 0.2128 - accuracy: 0.9362 - val_loss: 0.1057 - val_accuracy: 0.9719
Epoch 7/10
247/247 [==============================] - 126s 509ms/step - loss: 0.1695 - accuracy: 0.9500 - val_loss: 0.0444

In [28]:
n=9
print(n)#9

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

9
Epoch 1/10
274/274 [==============================] - 142s 504ms/step - loss: 3.5695 - accuracy: 0.0561 - val_loss: 3.2127 - val_accuracy: 0.1909
Epoch 2/10
274/274 [==============================] - 139s 509ms/step - loss: 2.3547 - accuracy: 0.3446 - val_loss: 0.4720 - val_accuracy: 0.8632
Epoch 3/10
274/274 [==============================] - 138s 505ms/step - loss: 0.6745 - accuracy: 0.7963 - val_loss: 0.1394 - val_accuracy: 0.9610
Epoch 4/10
274/274 [==============================] - 139s 506ms/step - loss: 0.3010 - accuracy: 0.9082 - val_loss: 0.0907 - val_accuracy: 0.9755
Epoch 5/10
274/274 [==============================] - 140s 510ms/step - loss: 0.1910 - accuracy: 0.9427 - val_loss: 0.0499 - val_accuracy: 0.9860
Epoch 6/10
274/274 [==============================] - 138s 503ms/step - loss: 0.1381 - accuracy: 0.9594 - val_loss: 0.0452 - val_accuracy: 0.9885
Epoch 7/10
274/274 [==============================] - 138s 503ms/step - loss: 0.1147 - accuracy: 0.9673 - val_loss: 0.0483

In [29]:
n=10
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

10
Epoch 1/10
301/301 [==============================] - 156s 502ms/step - loss: 3.5454 - accuracy: 0.0578 - val_loss: 2.6785 - val_accuracy: 0.2853
Epoch 2/10
301/301 [==============================] - 155s 515ms/step - loss: 2.1873 - accuracy: 0.3964 - val_loss: 0.3599 - val_accuracy: 0.8974
Epoch 3/10
301/301 [==============================] - 152s 505ms/step - loss: 0.6060 - accuracy: 0.8158 - val_loss: 0.1136 - val_accuracy: 0.9681
Epoch 4/10
301/301 [==============================] - 141s 468ms/step - loss: 0.2966 - accuracy: 0.9112 - val_loss: 0.0913 - val_accuracy: 0.9747
Epoch 5/10
301/301 [==============================] - 166s 550ms/step - loss: 0.1944 - accuracy: 0.9415 - val_loss: 0.0483 - val_accuracy: 0.9880
Epoch 6/10
301/301 [==============================] - 152s 504ms/step - loss: 0.1426 - accuracy: 0.9585 - val_loss: 0.0561 - val_accuracy: 0.9867
Epoch 7/10
301/301 [==============================] - 154s 511ms/step - loss: 0.1189 - accuracy: 0.9672 - val_loss: 0.059

In [30]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

11
Epoch 1/10
329/329 [==============================] - 168s 498ms/step - loss: 3.5664 - accuracy: 0.0593 - val_loss: 2.5647 - val_accuracy: 0.3279
Epoch 2/10
329/329 [==============================] - 165s 500ms/step - loss: 2.0284 - accuracy: 0.4494 - val_loss: 0.2920 - val_accuracy: 0.9041
Epoch 3/10
329/329 [==============================] - 165s 502ms/step - loss: 0.4959 - accuracy: 0.8479 - val_loss: 0.0732 - val_accuracy: 0.9750
Epoch 4/10
329/329 [==============================] - 164s 500ms/step - loss: 0.2307 - accuracy: 0.9342 - val_loss: 0.0582 - val_accuracy: 0.9862
Epoch 5/10
329/329 [==============================] - 164s 499ms/step - loss: 0.1569 - accuracy: 0.9537 - val_loss: 0.0378 - val_accuracy: 0.9890
Epoch 6/10
329/329 [==============================] - 164s 499ms/step - loss: 0.1178 - accuracy: 0.9656 - val_loss: 0.0349 - val_accuracy: 0.9934
Epoch 7/10
329/329 [==============================] - 164s 499ms/step - loss: 0.1012 - accuracy: 0.9697 - val_loss: 0.135

In [31]:
n=12
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

12
Epoch 1/10
356/356 [==============================] - 180s 495ms/step - loss: 3.5460 - accuracy: 0.0651 - val_loss: 2.0954 - val_accuracy: 0.4282
Epoch 2/10
356/356 [==============================] - 178s 499ms/step - loss: 1.6733 - accuracy: 0.5330 - val_loss: 0.2531 - val_accuracy: 0.9212
Epoch 3/10
356/356 [==============================] - 178s 499ms/step - loss: 0.4274 - accuracy: 0.8708 - val_loss: 0.0918 - val_accuracy: 0.9752
Epoch 4/10
356/356 [==============================] - 178s 500ms/step - loss: 0.2133 - accuracy: 0.9357 - val_loss: 0.0501 - val_accuracy: 0.9888
Epoch 5/10
356/356 [==============================] - 177s 498ms/step - loss: 0.1433 - accuracy: 0.9579 - val_loss: 0.0454 - val_accuracy: 0.9872
Epoch 6/10
356/356 [==============================] - 177s 496ms/step - loss: 0.1126 - accuracy: 0.9682 - val_loss: 0.0300 - val_accuracy: 0.9918
Epoch 7/10
356/356 [==============================] - 178s 499ms/step - loss: 0.1096 - accuracy: 0.9697 - val_loss: 0.029

In [32]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

13
Epoch 1/10
383/383 [==============================] - 194s 495ms/step - loss: 3.5085 - accuracy: 0.0678 - val_loss: 1.5365 - val_accuracy: 0.5542
Epoch 2/10
383/383 [==============================] - 191s 498ms/step - loss: 1.4205 - accuracy: 0.5835 - val_loss: 0.1881 - val_accuracy: 0.9449
Epoch 3/10
383/383 [==============================] - 191s 498ms/step - loss: 0.4002 - accuracy: 0.8797 - val_loss: 0.0803 - val_accuracy: 0.9770
Epoch 4/10
383/383 [==============================] - 191s 499ms/step - loss: 0.2021 - accuracy: 0.9400 - val_loss: 0.0542 - val_accuracy: 0.9844
Epoch 5/10
383/383 [==============================] - 190s 497ms/step - loss: 0.1363 - accuracy: 0.9603 - val_loss: 0.0425 - val_accuracy: 0.9888
Epoch 6/10
383/383 [==============================] - 192s 500ms/step - loss: 0.1173 - accuracy: 0.9657 - val_loss: 0.0302 - val_accuracy: 0.9939
Epoch 7/10
383/383 [==============================] - 190s 496ms/step - loss: 0.0999 - accuracy: 0.9715 - val_loss: 0.031

In [33]:
print(n)#14

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

14
Epoch 1/10
411/411 [==============================] - 207s 494ms/step - loss: 3.4142 - accuracy: 0.0986 - val_loss: 1.1170 - val_accuracy: 0.6466
Epoch 2/10
411/411 [==============================] - 205s 500ms/step - loss: 1.0106 - accuracy: 0.7023 - val_loss: 0.3515 - val_accuracy: 0.8903
Epoch 3/10
411/411 [==============================] - 205s 498ms/step - loss: 0.2771 - accuracy: 0.9189 - val_loss: 0.0630 - val_accuracy: 0.9821
Epoch 4/10
411/411 [==============================] - 204s 496ms/step - loss: 0.1639 - accuracy: 0.9509 - val_loss: 0.0409 - val_accuracy: 0.9890
Epoch 5/10
411/411 [==============================] - 204s 495ms/step - loss: 0.1244 - accuracy: 0.9647 - val_loss: 0.0335 - val_accuracy: 0.9911
Epoch 6/10
411/411 [==============================] - 204s 496ms/step - loss: 0.1001 - accuracy: 0.9709 - val_loss: 0.0389 - val_accuracy: 0.9926
Epoch 7/10
411/411 [==============================] - 206s 501ms/step - loss: 0.0831 - accuracy: 0.9765 - val_loss: 0.093

In [34]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

15
Epoch 1/10
438/438 [==============================] - 221s 494ms/step - loss: 3.4797 - accuracy: 0.0762 - val_loss: 1.4844 - val_accuracy: 0.5458
Epoch 2/10
438/438 [==============================] - 217s 496ms/step - loss: 1.2223 - accuracy: 0.6316 - val_loss: 0.1052 - val_accuracy: 0.9676
Epoch 3/10
438/438 [==============================] - 216s 494ms/step - loss: 0.2967 - accuracy: 0.9092 - val_loss: 0.0522 - val_accuracy: 0.9839
Epoch 4/10
438/438 [==============================] - 197s 450ms/step - loss: 0.1628 - accuracy: 0.9544 - val_loss: 0.0386 - val_accuracy: 0.9900
Epoch 5/10
438/438 [==============================] - 222s 506ms/step - loss: 0.1150 - accuracy: 0.9673 - val_loss: 0.0278 - val_accuracy: 0.9957
Epoch 6/10
438/438 [==============================] - 228s 520ms/step - loss: 0.1092 - accuracy: 0.9680 - val_loss: 0.0411 - val_accuracy: 0.9913
Epoch 7/10
438/438 [==============================] - 223s 509ms/step - loss: 0.0817 - accuracy: 0.9767 - val_loss: 0.031

In [35]:
n=16
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

16
Epoch 1/10
465/465 [==============================] - 233s 492ms/step - loss: 3.4779 - accuracy: 0.0798 - val_loss: 1.1071 - val_accuracy: 0.7002
Epoch 2/10
465/465 [==============================] - 230s 494ms/step - loss: 1.0484 - accuracy: 0.6953 - val_loss: 0.1525 - val_accuracy: 0.9538
Epoch 3/10
465/465 [==============================] - 229s 493ms/step - loss: 0.2912 - accuracy: 0.9147 - val_loss: 0.0417 - val_accuracy: 0.9883
Epoch 4/10
465/465 [==============================] - 232s 498ms/step - loss: 0.1481 - accuracy: 0.9581 - val_loss: 0.0330 - val_accuracy: 0.9916
Epoch 5/10
465/465 [==============================] - 216s 465ms/step - loss: 0.1183 - accuracy: 0.9670 - val_loss: 0.0378 - val_accuracy: 0.9911
Epoch 6/10
465/465 [==============================] - 197s 423ms/step - loss: 0.0902 - accuracy: 0.9738 - val_loss: 0.0258 - val_accuracy: 0.9936
Epoch 7/10
465/465 [==============================] - 198s 425ms/step - loss: 0.0870 - accuracy: 0.9750 - val_loss: 0.032

In [36]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

17
Epoch 1/10
493/493 [==============================] - 213s 424ms/step - loss: 3.3588 - accuracy: 0.1237 - val_loss: 0.6774 - val_accuracy: 0.8045
Epoch 2/10
493/493 [==============================] - 209s 424ms/step - loss: 0.7825 - accuracy: 0.7687 - val_loss: 0.1231 - val_accuracy: 0.9620
Epoch 3/10
493/493 [==============================] - 230s 466ms/step - loss: 0.2713 - accuracy: 0.9204 - val_loss: 0.0526 - val_accuracy: 0.9844
Epoch 4/10
493/493 [==============================] - 227s 461ms/step - loss: 0.1664 - accuracy: 0.9535 - val_loss: 0.0538 - val_accuracy: 0.9860
Epoch 5/10
493/493 [==============================] - 228s 462ms/step - loss: 0.1190 - accuracy: 0.9647 - val_loss: 0.0319 - val_accuracy: 0.9939
Epoch 6/10
493/493 [==============================] - 228s 462ms/step - loss: 0.1085 - accuracy: 0.9688 - val_loss: 0.0282 - val_accuracy: 0.9936
Epoch 7/10
493/493 [==============================] - 228s 462ms/step - loss: 0.0955 - accuracy: 0.9727 - val_loss: 0.023

In [37]:
n=18
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

18
Epoch 1/10
520/520 [==============================] - 246s 461ms/step - loss: 3.4529 - accuracy: 0.0837 - val_loss: 0.9779 - val_accuracy: 0.7405
Epoch 2/10
520/520 [==============================] - 240s 462ms/step - loss: 0.8663 - accuracy: 0.7419 - val_loss: 0.1577 - val_accuracy: 0.9582
Epoch 3/10
520/520 [==============================] - 240s 462ms/step - loss: 0.2418 - accuracy: 0.9286 - val_loss: 0.0831 - val_accuracy: 0.9775
Epoch 4/10
520/520 [==============================] - 240s 462ms/step - loss: 0.1396 - accuracy: 0.9607 - val_loss: 0.0475 - val_accuracy: 0.9860
Epoch 5/10
520/520 [==============================] - 240s 461ms/step - loss: 0.1051 - accuracy: 0.9681 - val_loss: 0.0554 - val_accuracy: 0.9872
Epoch 6/10
520/520 [==============================] - 239s 461ms/step - loss: 0.0894 - accuracy: 0.9748 - val_loss: 0.0358 - val_accuracy: 0.9898
Epoch 7/10
520/520 [==============================] - 239s 460ms/step - loss: 0.0816 - accuracy: 0.9766 - val_loss: 0.039

In [38]:
n=19
print(n)#19

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

19
Epoch 1/10
547/547 [==============================] - 257s 461ms/step - loss: 3.3034 - accuracy: 0.1245 - val_loss: 0.5306 - val_accuracy: 0.8456
Epoch 2/10
547/547 [==============================] - 255s 466ms/step - loss: 0.6385 - accuracy: 0.8132 - val_loss: 0.0748 - val_accuracy: 0.9809
Epoch 3/10
547/547 [==============================] - 253s 463ms/step - loss: 0.2249 - accuracy: 0.9362 - val_loss: 0.0523 - val_accuracy: 0.9880
Epoch 4/10
547/547 [==============================] - 254s 464ms/step - loss: 0.1366 - accuracy: 0.9595 - val_loss: 0.0256 - val_accuracy: 0.9934
Epoch 5/10
547/547 [==============================] - 254s 465ms/step - loss: 0.1105 - accuracy: 0.9681 - val_loss: 0.0222 - val_accuracy: 0.9941
Epoch 6/10
547/547 [==============================] - 253s 463ms/step - loss: 0.1045 - accuracy: 0.9723 - val_loss: 0.0215 - val_accuracy: 0.9959
Epoch 7/10
547/547 [==============================] - 253s 462ms/step - loss: 0.0889 - accuracy: 0.9759 - val_loss: 0.044

In [39]:

print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

20
Epoch 1/10
552/552 [==============================] - 259s 461ms/step - loss: 3.3355 - accuracy: 0.1086 - val_loss: 0.7901 - val_accuracy: 0.7423
Epoch 2/10
552/552 [==============================] - 255s 463ms/step - loss: 0.7054 - accuracy: 0.7880 - val_loss: 0.1537 - val_accuracy: 0.9604
Epoch 3/10
552/552 [==============================] - 255s 462ms/step - loss: 0.2240 - accuracy: 0.9334 - val_loss: 0.0327 - val_accuracy: 0.9939
Epoch 4/10
552/552 [==============================] - 254s 461ms/step - loss: 0.1389 - accuracy: 0.9603 - val_loss: 0.0403 - val_accuracy: 0.9921
Epoch 5/10
552/552 [==============================] - 255s 462ms/step - loss: 0.1081 - accuracy: 0.9689 - val_loss: 0.0193 - val_accuracy: 0.9964
Epoch 6/10
552/552 [==============================] - 257s 465ms/step - loss: 0.0961 - accuracy: 0.9745 - val_loss: 0.0373 - val_accuracy: 0.9949
Epoch 7/10
552/552 [==============================] - 254s 460ms/step - loss: 0.0848 - accuracy: 0.9760 - val_loss: 0.017

In [40]:
new_model_random_dir  = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_random_e1"

i=3

for model in models_random[3:]:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_random_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_random_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_random_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_random_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_random_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_random_e1_8\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_random_e1_9\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_random_e1_10\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/m

In [ ]:
loading = False

models_random = []

if loading:
    for i in range(20):
        model_random_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_random_e1_"+str(i)
        print(model_random_dir)
        model =utils.My_model('gtsrb',True,model_random_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_random.append(model)
       

In [149]:
try:
    del random_values
    del top_images_by_random
    del top_labels_by_random
    del image_sets_random
    del label_sets_random
    del models_random
except:
    print("Error")

Error


In [154]:
gc.collect()

6486

## Training guided by NC

In [92]:
# NC
nc_values = []
for i in range(1,13):
    #save_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/NC_values/nc_values_"+str(i)+".npy"
    save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/gtsrb_nc_values_"+str(i)+".npy"

    #print(save_dir_rand)
    tmp_values = np.load(save_dir)
    #print(tmp_values.shape)
    nc_values = np.append(nc_values,tmp_values)

In [93]:
nc_values.shape

(35287,)

In [94]:

top_images_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),x_train_and_adversary)
top_labels_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),y_train_and_adversary)


In [95]:
m = n_data_points
n = 0
image_sets_nc = []
label_sets_nc = []


for i in range((len(top_images_by_nc)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_nc)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_nc)%m))
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m+(len(top_images_by_nc)%m)])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m+(len(top_images_by_nc)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m])
    image_sets_nc.append(top_images_by_nc_n)
    label_sets_nc.append(top_labels_by_nc_n)
    print(len(top_images_by_nc_n))
    n += m




0 :
0  ->  1750
1750
1 :
0  ->  3500
3500
2 :
0  ->  5250
5250
3 :
0  ->  7000
7000
4 :
0  ->  8750
8750
5 :
0  ->  10500
10500
6 :
0  ->  12250
12250
7 :
0  ->  14000
14000
8 :
0  ->  15750
15750
9 :
0  ->  17500
17500
10 :
0  ->  19250
19250
11 :
0  ->  21000
21000
12 :
0  ->  22750
22750
13 :
0  ->  24500
24500
14 :
0  ->  26250
26250
15 :
0  ->  28000
28000
16 :
0  ->  29750
29750
17 :
0  ->  31500
31500
18 :
0  ->  33250
33250
19 :
0  ->  35000
35000
20 :
Last
0  ->  37037
35287


In [96]:
print(model_dir)

models_nc = []
for i in range(len(label_sets_nc)):
    print(i,":")
    model = utils.My_model(dataset,False,None)
    model.compile_model()
    models_nc.append(model)


C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/
0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled
20 :
Model compiled


In [97]:
n=0
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

0
Epoch 1/10
28/28 [==============================] - 39s 1s/step - loss: 2.8898 - accuracy: 0.4021 - val_loss: 8.1936 - val_accuracy: 0.0574
Epoch 2/10
28/28 [==============================] - 28s 996ms/step - loss: 1.1312 - accuracy: 0.4507 - val_loss: 8.9305 - val_accuracy: 0.0620
Epoch 3/10
28/28 [==============================] - 27s 989ms/step - loss: 1.0065 - accuracy: 0.4907 - val_loss: 6.9683 - val_accuracy: 0.0947
Epoch 4/10
28/28 [==============================] - 28s 1s/step - loss: 0.6759 - accuracy: 0.7294 - val_loss: 12.5758 - val_accuracy: 0.1135
Epoch 5/10
28/28 [==============================] - 28s 1s/step - loss: 0.2722 - accuracy: 0.9160 - val_loss: 11.5658 - val_accuracy: 0.1276
Epoch 6/10
28/28 [==============================] - 28s 1000ms/step - loss: 0.2127 - accuracy: 0.9190 - val_loss: 12.1767 - val_accuracy: 0.1312
Epoch 7/10
28/28 [==============================] - 29s 1s/step - loss: 0.1435 - accuracy: 0.9522 - val_loss: 13.5869 - val_accuracy: 0.1424
Epoc

In [98]:

print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

1
Epoch 1/10
55/55 [==============================] - 56s 836ms/step - loss: 1.8409 - accuracy: 0.3812 - val_loss: 7.2774 - val_accuracy: 0.0646
Epoch 2/10
55/55 [==============================] - 45s 817ms/step - loss: 1.0537 - accuracy: 0.4788 - val_loss: 13.1253 - val_accuracy: 0.1084
Epoch 3/10
55/55 [==============================] - 46s 840ms/step - loss: 0.4133 - accuracy: 0.8418 - val_loss: 8.7736 - val_accuracy: 0.1240
Epoch 4/10
55/55 [==============================] - 45s 830ms/step - loss: 0.2354 - accuracy: 0.9221 - val_loss: 13.9278 - val_accuracy: 0.1434
Epoch 5/10
55/55 [==============================] - 33s 591ms/step - loss: 0.1253 - accuracy: 0.9692 - val_loss: 13.0958 - val_accuracy: 0.1582
Epoch 6/10
55/55 [==============================] - 30s 554ms/step - loss: 0.0726 - accuracy: 0.9855 - val_loss: 16.1308 - val_accuracy: 0.1579
Epoch 7/10
55/55 [==============================] - 30s 554ms/step - loss: 0.0435 - accuracy: 0.9901 - val_loss: 15.8347 - val_accuracy:

In [99]:

print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

2
Epoch 1/10
83/83 [==============================] - 49s 536ms/step - loss: 1.9321 - accuracy: 0.3238 - val_loss: 9.2605 - val_accuracy: 0.0681
Epoch 2/10
83/83 [==============================] - 45s 538ms/step - loss: 1.2417 - accuracy: 0.4902 - val_loss: 9.5498 - val_accuracy: 0.1266
Epoch 3/10
83/83 [==============================] - 46s 554ms/step - loss: 0.6356 - accuracy: 0.7823 - val_loss: 4.9545 - val_accuracy: 0.1266
Epoch 4/10
83/83 [==============================] - 47s 572ms/step - loss: 0.4374 - accuracy: 0.8842 - val_loss: 14.4051 - val_accuracy: 0.1567
Epoch 5/10
83/83 [==============================] - 47s 563ms/step - loss: 0.2524 - accuracy: 0.9321 - val_loss: 6.3458 - val_accuracy: 0.1600
Epoch 6/10
83/83 [==============================] - 47s 568ms/step - loss: 0.2036 - accuracy: 0.9446 - val_loss: 13.2238 - val_accuracy: 0.1457
Epoch 7/10
83/83 [==============================] - 47s 570ms/step - loss: 0.1975 - accuracy: 0.9514 - val_loss: 10.4364 - val_accuracy: 0

In [100]:

print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

3
Epoch 1/10
110/110 [==============================] - 65s 550ms/step - loss: 2.3125 - accuracy: 0.2534 - val_loss: 4.7377 - val_accuracy: 0.0755
Epoch 2/10
110/110 [==============================] - 60s 544ms/step - loss: 1.7010 - accuracy: 0.4026 - val_loss: 5.9562 - val_accuracy: 0.1312
Epoch 3/10
110/110 [==============================] - 60s 547ms/step - loss: 1.0825 - accuracy: 0.6405 - val_loss: 7.8958 - val_accuracy: 0.2001
Epoch 4/10
110/110 [==============================] - 60s 550ms/step - loss: 0.6365 - accuracy: 0.7967 - val_loss: 7.3251 - val_accuracy: 0.1870
Epoch 5/10
110/110 [==============================] - 60s 548ms/step - loss: 0.3907 - accuracy: 0.8788 - val_loss: 9.4240 - val_accuracy: 0.3782
Epoch 6/10
110/110 [==============================] - 60s 545ms/step - loss: 0.2431 - accuracy: 0.9311 - val_loss: 7.2663 - val_accuracy: 0.4348
Epoch 7/10
110/110 [==============================] - 60s 550ms/step - loss: 0.1636 - accuracy: 0.9499 - val_loss: 7.9070 - val_

In [101]:
#4
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

4
Epoch 1/10
137/137 [==============================] - 79s 545ms/step - loss: 2.5450 - accuracy: 0.2175 - val_loss: 5.3183 - val_accuracy: 0.0952
Epoch 2/10
137/137 [==============================] - 76s 553ms/step - loss: 1.7156 - accuracy: 0.4302 - val_loss: 8.1487 - val_accuracy: 0.2965
Epoch 3/10
137/137 [==============================] - 74s 538ms/step - loss: 0.7308 - accuracy: 0.7659 - val_loss: 10.5233 - val_accuracy: 0.4177
Epoch 4/10
137/137 [==============================] - 75s 546ms/step - loss: 0.2562 - accuracy: 0.9248 - val_loss: 4.7467 - val_accuracy: 0.5063
Epoch 5/10
137/137 [==============================] - 74s 541ms/step - loss: 0.1473 - accuracy: 0.9607 - val_loss: 4.9063 - val_accuracy: 0.5162
Epoch 6/10
137/137 [==============================] - 74s 537ms/step - loss: 0.1171 - accuracy: 0.9673 - val_loss: 8.9874 - val_accuracy: 0.5233
Epoch 7/10
137/137 [==============================] - 74s 538ms/step - loss: 0.0939 - accuracy: 0.9752 - val_loss: 10.2453 - va

In [102]:
n=5
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

5
Epoch 1/10
165/165 [==============================] - 92s 529ms/step - loss: 2.8615 - accuracy: 0.1662 - val_loss: 8.1425 - val_accuracy: 0.0804
Epoch 2/10
165/165 [==============================] - 87s 526ms/step - loss: 1.7584 - accuracy: 0.4120 - val_loss: 9.3901 - val_accuracy: 0.3312
Epoch 3/10
165/165 [==============================] - 88s 532ms/step - loss: 0.5916 - accuracy: 0.8143 - val_loss: 6.4114 - val_accuracy: 0.4578
Epoch 4/10
165/165 [==============================] - 89s 537ms/step - loss: 0.2169 - accuracy: 0.9384 - val_loss: 9.9784 - val_accuracy: 0.5285
Epoch 5/10
165/165 [==============================] - 88s 533ms/step - loss: 0.1286 - accuracy: 0.9652 - val_loss: 10.2124 - val_accuracy: 0.5152
Epoch 6/10
165/165 [==============================] - 95s 579ms/step - loss: 0.1003 - accuracy: 0.9737 - val_loss: 9.6486 - val_accuracy: 0.5527
Epoch 7/10
165/165 [==============================] - 86s 521ms/step - loss: 0.0851 - accuracy: 0.9791 - val_loss: 9.0275 - val

In [103]:
n=6
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

6
Epoch 1/10
192/192 [==============================] - 101s 503ms/step - loss: 3.1940 - accuracy: 0.1470 - val_loss: 6.1583 - val_accuracy: 0.0786
Epoch 2/10
192/192 [==============================] - 96s 499ms/step - loss: 1.6740 - accuracy: 0.4253 - val_loss: 4.7386 - val_accuracy: 0.4343
Epoch 3/10
192/192 [==============================] - 96s 502ms/step - loss: 0.3719 - accuracy: 0.8934 - val_loss: 9.1510 - val_accuracy: 0.5142
Epoch 4/10
192/192 [==============================] - 100s 520ms/step - loss: 0.1769 - accuracy: 0.9495 - val_loss: 9.8147 - val_accuracy: 0.5438
Epoch 5/10
192/192 [==============================] - 100s 521ms/step - loss: 0.1214 - accuracy: 0.9681 - val_loss: 5.6743 - val_accuracy: 0.5448
Epoch 6/10
192/192 [==============================] - 100s 520ms/step - loss: 0.0871 - accuracy: 0.9755 - val_loss: 11.3790 - val_accuracy: 0.5410
Epoch 7/10
192/192 [==============================] - 104s 541ms/step - loss: 0.0786 - accuracy: 0.9798 - val_loss: 8.7430 

In [104]:
n=7
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

7
Epoch 1/10
219/219 [==============================] - 118s 517ms/step - loss: 2.7249 - accuracy: 0.1260 - val_loss: 5.6114 - val_accuracy: 0.1990
Epoch 2/10
219/219 [==============================] - 114s 519ms/step - loss: 1.3056 - accuracy: 0.5521 - val_loss: 6.0110 - val_accuracy: 0.5027
Epoch 3/10
219/219 [==============================] - 113s 518ms/step - loss: 0.3124 - accuracy: 0.9118 - val_loss: 7.6723 - val_accuracy: 0.5471
Epoch 4/10
219/219 [==============================] - 114s 520ms/step - loss: 0.1502 - accuracy: 0.9596 - val_loss: 6.4933 - val_accuracy: 0.5542
Epoch 5/10
219/219 [==============================] - 113s 517ms/step - loss: 0.1030 - accuracy: 0.9735 - val_loss: 5.6000 - val_accuracy: 0.5555
Epoch 6/10
219/219 [==============================] - 114s 519ms/step - loss: 0.0771 - accuracy: 0.9793 - val_loss: 10.1448 - val_accuracy: 0.5547
Epoch 7/10
219/219 [==============================] - 113s 518ms/step - loss: 0.0755 - accuracy: 0.9804 - val_loss: 9.439

In [105]:
n=8
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

8
Epoch 1/10
247/247 [==============================] - 132s 516ms/step - loss: 2.7474 - accuracy: 0.1364 - val_loss: 3.6525 - val_accuracy: 0.1937
Epoch 2/10
247/247 [==============================] - 128s 518ms/step - loss: 1.0898 - accuracy: 0.6382 - val_loss: 3.0780 - val_accuracy: 0.4960
Epoch 3/10
247/247 [==============================] - 128s 518ms/step - loss: 0.2464 - accuracy: 0.9341 - val_loss: 5.7663 - val_accuracy: 0.5558
Epoch 4/10
247/247 [==============================] - 128s 517ms/step - loss: 0.1469 - accuracy: 0.9639 - val_loss: 10.8314 - val_accuracy: 0.5088
Epoch 5/10
247/247 [==============================] - 128s 518ms/step - loss: 0.1359 - accuracy: 0.9678 - val_loss: 5.7412 - val_accuracy: 0.5808
Epoch 6/10
247/247 [==============================] - 128s 519ms/step - loss: 0.0948 - accuracy: 0.9754 - val_loss: 5.0574 - val_accuracy: 0.5879
Epoch 7/10
247/247 [==============================] - 128s 517ms/step - loss: 0.0797 - accuracy: 0.9781 - val_loss: 5.215

In [106]:
#reiniciar la 9
print(n)#9

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

9
Epoch 1/10
274/274 [==============================] - 146s 517ms/step - loss: 2.8771 - accuracy: 0.1218 - val_loss: 5.0074 - val_accuracy: 0.3003
Epoch 2/10
274/274 [==============================] - 142s 520ms/step - loss: 1.0655 - accuracy: 0.6536 - val_loss: 3.4228 - val_accuracy: 0.5723
Epoch 3/10
274/274 [==============================] - 143s 522ms/step - loss: 0.2309 - accuracy: 0.9364 - val_loss: 3.6279 - val_accuracy: 0.6338
Epoch 4/10
274/274 [==============================] - 143s 523ms/step - loss: 0.1494 - accuracy: 0.9628 - val_loss: 6.6400 - val_accuracy: 0.6930
Epoch 5/10
274/274 [==============================] - 142s 517ms/step - loss: 0.1040 - accuracy: 0.9748 - val_loss: 4.2406 - val_accuracy: 0.7180
Epoch 6/10
274/274 [==============================] - 141s 516ms/step - loss: 0.0838 - accuracy: 0.9801 - val_loss: 4.1200 - val_accuracy: 0.7129
Epoch 7/10
274/274 [==============================] - 142s 517ms/step - loss: 0.0841 - accuracy: 0.9781 - val_loss: 5.3346

In [107]:
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

10
Epoch 1/10
301/301 [==============================] - 177s 562ms/step - loss: 2.8452 - accuracy: 0.1213 - val_loss: 4.8083 - val_accuracy: 0.3702
Epoch 2/10
301/301 [==============================] - 162s 537ms/step - loss: 0.9515 - accuracy: 0.6981 - val_loss: 4.9266 - val_accuracy: 0.6915
Epoch 3/10
301/301 [==============================] - 162s 537ms/step - loss: 0.2295 - accuracy: 0.9408 - val_loss: 6.8583 - val_accuracy: 0.7224
Epoch 4/10
301/301 [==============================] - 158s 526ms/step - loss: 0.1308 - accuracy: 0.9663 - val_loss: 5.4077 - val_accuracy: 0.7420
Epoch 5/10
301/301 [==============================] - 157s 522ms/step - loss: 0.1040 - accuracy: 0.9744 - val_loss: 6.2442 - val_accuracy: 0.7499
Epoch 6/10
301/301 [==============================] - 157s 522ms/step - loss: 0.0934 - accuracy: 0.9821 - val_loss: 7.0191 - val_accuracy: 0.7487
Epoch 7/10
301/301 [==============================] - 155s 517ms/step - loss: 0.0765 - accuracy: 0.9832 - val_loss: 5.923

In [108]:
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

11
Epoch 1/10
329/329 [==============================] - 174s 517ms/step - loss: 2.8425 - accuracy: 0.1329 - val_loss: 4.9431 - val_accuracy: 0.4072
Epoch 2/10
329/329 [==============================] - 172s 524ms/step - loss: 0.8701 - accuracy: 0.7321 - val_loss: 6.3606 - val_accuracy: 0.7295
Epoch 3/10
329/329 [==============================] - 171s 520ms/step - loss: 0.2089 - accuracy: 0.9474 - val_loss: 3.2567 - val_accuracy: 0.7359
Epoch 4/10
329/329 [==============================] - 170s 517ms/step - loss: 0.1195 - accuracy: 0.9693 - val_loss: 3.9127 - val_accuracy: 0.7487
Epoch 5/10
329/329 [==============================] - 170s 517ms/step - loss: 0.0981 - accuracy: 0.9743 - val_loss: 5.1844 - val_accuracy: 0.7635
Epoch 6/10
329/329 [==============================] - 170s 517ms/step - loss: 0.0952 - accuracy: 0.9757 - val_loss: 6.6183 - val_accuracy: 0.7719
Epoch 7/10
329/329 [==============================] - 170s 517ms/step - loss: 0.0760 - accuracy: 0.9803 - val_loss: 8.156

In [109]:
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

12
Epoch 1/10
356/356 [==============================] - 189s 518ms/step - loss: 3.0030 - accuracy: 0.1357 - val_loss: 6.1479 - val_accuracy: 0.6035
Epoch 2/10
356/356 [==============================] - 184s 517ms/step - loss: 0.6098 - accuracy: 0.8183 - val_loss: 2.4592 - val_accuracy: 0.7665
Epoch 3/10
356/356 [==============================] - 184s 517ms/step - loss: 0.1723 - accuracy: 0.9570 - val_loss: 3.4790 - val_accuracy: 0.7999
Epoch 4/10
356/356 [==============================] - 183s 515ms/step - loss: 0.1166 - accuracy: 0.9715 - val_loss: 5.3748 - val_accuracy: 0.8221
Epoch 5/10
356/356 [==============================] - 184s 516ms/step - loss: 0.0838 - accuracy: 0.9810 - val_loss: 5.9516 - val_accuracy: 0.8288
Epoch 6/10
356/356 [==============================] - 184s 516ms/step - loss: 0.0729 - accuracy: 0.9810 - val_loss: 6.0545 - val_accuracy: 0.8216
Epoch 7/10
356/356 [==============================] - 184s 516ms/step - loss: 0.0650 - accuracy: 0.9851 - val_loss: 3.836

In [110]:
n=13
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

13
Epoch 1/10
383/383 [==============================] - 204s 518ms/step - loss: 3.1827 - accuracy: 0.1150 - val_loss: 3.7307 - val_accuracy: 0.5609
Epoch 2/10
383/383 [==============================] - 200s 522ms/step - loss: 0.7264 - accuracy: 0.7778 - val_loss: 3.3003 - val_accuracy: 0.7874
Epoch 3/10
383/383 [==============================] - 199s 521ms/step - loss: 0.2286 - accuracy: 0.9458 - val_loss: 3.9308 - val_accuracy: 0.8267
Epoch 4/10
383/383 [==============================] - 198s 518ms/step - loss: 0.1269 - accuracy: 0.9689 - val_loss: 4.6978 - val_accuracy: 0.8331
Epoch 5/10
383/383 [==============================] - 198s 518ms/step - loss: 0.1073 - accuracy: 0.9749 - val_loss: 2.8663 - val_accuracy: 0.8130
Epoch 6/10
383/383 [==============================] - 198s 517ms/step - loss: 0.0828 - accuracy: 0.9786 - val_loss: 3.0880 - val_accuracy: 0.8395
Epoch 7/10
383/383 [==============================] - 198s 517ms/step - loss: 0.0746 - accuracy: 0.9820 - val_loss: 5.496

In [111]:
#14
n=14
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

14
Epoch 1/10
411/411 [==============================] - 218s 521ms/step - loss: 3.0432 - accuracy: 0.1456 - val_loss: 2.9755 - val_accuracy: 0.7170
Epoch 2/10
411/411 [==============================] - 214s 522ms/step - loss: 0.4789 - accuracy: 0.8691 - val_loss: 2.9863 - val_accuracy: 0.8298
Epoch 3/10
411/411 [==============================] - 214s 522ms/step - loss: 0.1592 - accuracy: 0.9620 - val_loss: 1.9737 - val_accuracy: 0.8329
Epoch 4/10
411/411 [==============================] - 214s 521ms/step - loss: 0.1051 - accuracy: 0.9748 - val_loss: 2.8013 - val_accuracy: 0.8441
Epoch 5/10
411/411 [==============================] - 214s 521ms/step - loss: 0.0984 - accuracy: 0.9777 - val_loss: 2.4237 - val_accuracy: 0.8502
Epoch 6/10
411/411 [==============================] - 216s 527ms/step - loss: 0.0892 - accuracy: 0.9792 - val_loss: 3.6380 - val_accuracy: 0.8533
Epoch 7/10
411/411 [==============================] - 214s 520ms/step - loss: 0.0858 - accuracy: 0.9789 - val_loss: 4.186

In [112]:
time.sleep(180)

In [113]:

print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

15
Epoch 1/10
438/438 [==============================] - 222s 496ms/step - loss: 3.1994 - accuracy: 0.1167 - val_loss: 3.0637 - val_accuracy: 0.6849
Epoch 2/10
438/438 [==============================] - 234s 533ms/step - loss: 0.6591 - accuracy: 0.8211 - val_loss: 1.9588 - val_accuracy: 0.8392
Epoch 3/10
438/438 [==============================] - 236s 539ms/step - loss: 0.1920 - accuracy: 0.9543 - val_loss: 2.0220 - val_accuracy: 0.8497
Epoch 4/10
438/438 [==============================] - 227s 518ms/step - loss: 0.1268 - accuracy: 0.9711 - val_loss: 2.0482 - val_accuracy: 0.8643
Epoch 5/10
438/438 [==============================] - 230s 524ms/step - loss: 0.1162 - accuracy: 0.9733 - val_loss: 3.3673 - val_accuracy: 0.8778
Epoch 6/10
438/438 [==============================] - 226s 517ms/step - loss: 0.0942 - accuracy: 0.9774 - val_loss: 1.9529 - val_accuracy: 0.8806
Epoch 7/10
438/438 [==============================] - 227s 518ms/step - loss: 0.0945 - accuracy: 0.9773 - val_loss: 2.448

In [114]:
time.sleep(180)

In [115]:
n=16
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

16
Epoch 1/10
465/465 [==============================] - 249s 526ms/step - loss: 3.1819 - accuracy: 0.1107 - val_loss: 2.2403 - val_accuracy: 0.7050
Epoch 2/10
465/465 [==============================] - 251s 540ms/step - loss: 0.6530 - accuracy: 0.8192 - val_loss: 1.5529 - val_accuracy: 0.8933
Epoch 3/10
465/465 [==============================] - 247s 530ms/step - loss: 0.1968 - accuracy: 0.9471 - val_loss: 1.3727 - val_accuracy: 0.9199
Epoch 4/10
465/465 [==============================] - 247s 530ms/step - loss: 0.1401 - accuracy: 0.9644 - val_loss: 1.4427 - val_accuracy: 0.9296
Epoch 5/10
465/465 [==============================] - 246s 530ms/step - loss: 0.1082 - accuracy: 0.9715 - val_loss: 1.2549 - val_accuracy: 0.9324
Epoch 6/10
465/465 [==============================] - 247s 532ms/step - loss: 0.0948 - accuracy: 0.9758 - val_loss: 1.4314 - val_accuracy: 0.9314
Epoch 7/10
465/465 [==============================] - 246s 529ms/step - loss: 0.0906 - accuracy: 0.9773 - val_loss: 1.955

In [116]:
n=17
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1 

17
Epoch 1/10
493/493 [==============================] - 264s 526ms/step - loss: 3.2945 - accuracy: 0.0946 - val_loss: 1.9581 - val_accuracy: 0.5764
Epoch 2/10
493/493 [==============================] - 262s 532ms/step - loss: 0.8118 - accuracy: 0.7675 - val_loss: 2.1105 - val_accuracy: 0.8694
Epoch 3/10
493/493 [==============================] - 261s 530ms/step - loss: 0.2229 - accuracy: 0.9400 - val_loss: 1.8059 - val_accuracy: 0.9291
Epoch 4/10
493/493 [==============================] - 260s 526ms/step - loss: 0.1468 - accuracy: 0.9604 - val_loss: 1.0766 - val_accuracy: 0.9510
Epoch 5/10
493/493 [==============================] - 260s 526ms/step - loss: 0.1093 - accuracy: 0.9705 - val_loss: 1.7042 - val_accuracy: 0.9492
Epoch 6/10
493/493 [==============================] - 264s 535ms/step - loss: 0.0944 - accuracy: 0.9747 - val_loss: 2.4801 - val_accuracy: 0.9510
Epoch 7/10
493/493 [==============================] - 261s 529ms/step - loss: 0.0893 - accuracy: 0.9784 - val_loss: 2.368

In [117]:
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

18
Epoch 1/10
520/520 [==============================] - 277s 525ms/step - loss: 3.3055 - accuracy: 0.1011 - val_loss: 1.4831 - val_accuracy: 0.6683
Epoch 2/10
520/520 [==============================] - 274s 528ms/step - loss: 0.7583 - accuracy: 0.7733 - val_loss: 1.0005 - val_accuracy: 0.9434
Epoch 3/10
520/520 [==============================] - 276s 531ms/step - loss: 0.2124 - accuracy: 0.9400 - val_loss: 0.7849 - val_accuracy: 0.9513
Epoch 4/10
520/520 [==============================] - 273s 526ms/step - loss: 0.1325 - accuracy: 0.9645 - val_loss: 0.7746 - val_accuracy: 0.9582
Epoch 5/10
520/520 [==============================] - 273s 525ms/step - loss: 0.1056 - accuracy: 0.9713 - val_loss: 0.9962 - val_accuracy: 0.9525
Epoch 6/10
520/520 [==============================] - 273s 526ms/step - loss: 0.0964 - accuracy: 0.9758 - val_loss: 0.6761 - val_accuracy: 0.9564
Epoch 7/10
520/520 [==============================] - 274s 528ms/step - loss: 0.0869 - accuracy: 0.9771 - val_loss: 0.781

In [120]:
n=19
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

19
Epoch 1/10
547/547 [==============================] - 253s 457ms/step - loss: 0.1609 - accuracy: 0.9611 - val_loss: 0.0272 - val_accuracy: 0.9941
Epoch 2/10
547/547 [==============================] - 253s 463ms/step - loss: 0.1422 - accuracy: 0.9642 - val_loss: 0.0270 - val_accuracy: 0.9949
Epoch 3/10
547/547 [==============================] - 267s 488ms/step - loss: 0.1252 - accuracy: 0.9686 - val_loss: 0.0179 - val_accuracy: 0.9954
Epoch 4/10
547/547 [==============================] - 282s 516ms/step - loss: 0.1239 - accuracy: 0.9712 - val_loss: 0.0594 - val_accuracy: 0.9857
Epoch 5/10
547/547 [==============================] - 283s 517ms/step - loss: 0.1255 - accuracy: 0.9704 - val_loss: 0.0220 - val_accuracy: 0.9949
Epoch 6/10
547/547 [==============================] - 282s 516ms/step - loss: 0.1356 - accuracy: 0.9690 - val_loss: 0.0386 - val_accuracy: 0.9944
Epoch 7/10
547/547 [==============================] - 282s 516ms/step - loss: 0.1361 - accuracy: 0.9693 - val_loss: 0.026

In [121]:
n=20
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

20
Epoch 1/10
552/552 [==============================] - 297s 529ms/step - loss: 3.2910 - accuracy: 0.1303 - val_loss: 0.7808 - val_accuracy: 0.7591
Epoch 2/10
552/552 [==============================] - 300s 544ms/step - loss: 0.5593 - accuracy: 0.8335 - val_loss: 0.1318 - val_accuracy: 0.9589
Epoch 3/10
552/552 [==============================] - 289s 524ms/step - loss: 0.1856 - accuracy: 0.9479 - val_loss: 0.0337 - val_accuracy: 0.9923
Epoch 4/10
552/552 [==============================] - 284s 514ms/step - loss: 0.1229 - accuracy: 0.9654 - val_loss: 0.0295 - val_accuracy: 0.9929
Epoch 5/10
552/552 [==============================] - 284s 514ms/step - loss: 0.0920 - accuracy: 0.9740 - val_loss: 0.0288 - val_accuracy: 0.9944
Epoch 6/10
552/552 [==============================] - 283s 513ms/step - loss: 0.0822 - accuracy: 0.9779 - val_loss: 0.0369 - val_accuracy: 0.9939
Epoch 7/10
552/552 [==============================] - 286s 517ms/step - loss: 0.0794 - accuracy: 0.9806 - val_loss: 0.030

In [122]:
new_model_nc_dir  = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_nc_e1"

i=19

for model in models_nc[19:]:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_nc_e1_19\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_nc_e1_20\assets
Model has been saved


In [ ]:
loading = False

models_nc = []

if loading:
    for i in range(20):
        model_nc_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_nc_e1"+str(i)
        print(model_nc_dir)
        model =utils.My_model(dataset,True,model_nc_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_nc.append(model)
       

In [123]:
try:
    del nc_values
    del top_images_by_nc
    del top_labels_by_nc
    del image_sets_nc
    del label_sets_nc
    del models_nc
except:
    print("Error")

In [124]:
gc.collect()

110759

## Evaluating

In [ ]:
model_original.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
# Metrics using adversarial test
evaluate_lsa_0 = []
evaluate_dsa_0 = []
evaluate_nc_0 = []
evaluate_random_0 = []

evaluate_lsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_dsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_nc_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_random_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

for model in models_lsa:
    evaluate_lsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_dsa:
    evaluate_dsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_random:
    evaluate_random_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_nc:
    evaluate_nc_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

In [ ]:
evaluate_lsa_0[0]=[0,0,0,0]
evaluate_dsa_0[0]=[0,0,0,0]
evaluate_random_0[0]=[0,0,0,0]
evaluate_nc_0[0]=[0,0,0,0]

In [ ]:
print(len(evaluate_lsa_0))
print(len(evaluate_dsa_0))
print(len(evaluate_random_0))
print(len(evaluate_nc_0))

In [ ]:
# Metrics using original test
evaluate_lsa_1 = []
evaluate_dsa_1 = []
evaluate_nc_1 = []
evaluate_random_1 = []

evaluate_lsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_dsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_nc_1.append(model_original.evaluate(x_test,y_test))
evaluate_random_1.append(model_original.evaluate(x_test,y_test))
print("lsa---")
for model in models_lsa:
    evaluate_lsa_1.append(model.evaluate(x_test,y_test))

print("dsa---")
    
for model in models_dsa:
    evaluate_dsa_1.append(model.evaluate(x_test,y_test))
    

print("random---")

for model in models_random:
    evaluate_random_1.append(model.evaluate(x_test,y_test))
    
print("nc---")

for model in models_nc:
    evaluate_nc_1.append(model.evaluate(x_test,y_test))


In [ ]:
evaluate_lsa_1[0]=[0,0,0,0]
evaluate_dsa_1[0]=[0,0,0,0]
evaluate_random_1[0]=[0,0,0,0]
evaluate_nc_1[0]=[0,0,0,0]

In [ ]:

print(len(evaluate_lsa_1))
print(len(evaluate_dsa_1))
print(len(evaluate_random_1))
print(len(evaluate_nc_1))

In [ ]:
type(evaluate_dsa_0[0][0])

In [ ]:
import pandas as pd

df_evaluate_lsa_0 = pd.DataFrame(np.array(evaluate_lsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_lsa_1 = pd.DataFrame(np.array(evaluate_lsa_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_0 = pd.DataFrame(np.array(evaluate_dsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_1 = pd.DataFrame(np.array(evaluate_dsa_1),columns=["loss","accuracy","precision","recall"])

df_evaluate_random_0 = pd.DataFrame(np.array(evaluate_random_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_random_1 = pd.DataFrame(np.array(evaluate_random_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_0 = pd.DataFrame(np.array(evaluate_nc_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_1 = pd.DataFrame(np.array(evaluate_nc_1),columns=["loss","accuracy","precision","recall"])


In [ ]:
# Original test set + adversarial set

metric ="accuracy"
accuracy_lsa_3 = (np.array(df_evaluate_lsa_0[metric])+np.array(df_evaluate_lsa_1[metric]))/2
accuracy_dsa_3 = (np.array(df_evaluate_dsa_0[metric])+np.array(df_evaluate_dsa_1[metric]))/2
accuracy_nc_3 = (np.array(df_evaluate_nc_0[metric])+np.array(df_evaluate_nc_1[metric]))/2
accuracy_random_3 = (np.array(df_evaluate_random_0[metric])+np.array(df_evaluate_random_1[metric]))/2


## Charts

In [ ]:
n_inputs = [1800*i for i in range(20)]
n_inputs.append(len(x_train_and_adversary))

In [ ]:
print(n_inputs)

In [ ]:
cd "C:/Users/fjdur/Desktop/gtsrb_graphs/e2_graphs_dec/"

In [ ]:
linestyles = ['solid','dotted','dashed','dashdot']
colors =['k','k','k','k']

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

#metric = "accuracy" # accuracy loss
my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_0[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_0[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_0[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_0[metric],colors[3],linestyle=linestyles[3])


    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with test set of adversarial examples FGSM")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_0.svg")
    plt.show()

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    
#metric = "accuracy" # accuracy loss
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_1[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_1[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_1[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_1[metric],colors[3],linestyle=linestyles[3])

    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with original set")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_1.svg")
    plt.show()

In [ ]:
#adversarial jsma test set
#configuration 3
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.plot(n_inputs,accuracy_lsa_3,colors[0],linestyle=linestyles[0])
plt.plot(n_inputs,accuracy_dsa_3,colors[1],linestyle=linestyles[1])
plt.plot(n_inputs,accuracy_random_3,colors[2],linestyle=linestyles[2])
plt.plot(n_inputs,accuracy_nc_3,colors[3],linestyle=linestyles[3])


legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



plt.legend(handles=legend_elements)#
plt.title("accuracy with both sets")

print(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max())
print(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max())
print(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max())
print(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max())

plt.plot(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max(),'-kD')
plt.plot(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max(),'-ko')
plt.plot(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max(),'-kv')
plt.plot(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max(),'-kp')

plt.xlabel('number of inputs')
plt.ylabel('accuracy')
plt.xlim([0, 37000])
#plt.ylim([0, 1])

plt.savefig("gtsrb_c1_"+"accuracy" + "_both.svg")

plt.show()

## Saving models

In [ ]:
new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_dsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2"

i=0
for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_random_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_random_e2"

i=0
for model in models_random:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_nc_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_nc_e2"

i=0
for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1